# Xgb 

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# %cd /content/drive/MyDrive/

In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 8623120763631442405
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9399762944
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 16008922424031325865
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:22:00.0, compute capability: 7.5"
 xla_global_id: 416903419,
 name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 1745276110
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 16519962093802775750
 physical_device_desc: "device: 1, name: NVIDIA RTX A2000 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
 xla_global_id: 2144165316]

In [4]:
import torch
# GPUの名称、CUDA Compute Capabilityを取得
print(torch.cuda.get_device_name())

NVIDIA RTX A2000 Laptop GPU


In [5]:
import cupy as cp
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import scipy.stats as st
import pickle
import optuna
import re
import gc

from lightgbm import LGBMClassifier

from sklearn import metrics
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.impute import SimpleImputer
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.metrics import roc_auc_score
from sklearn.datasets import make_moons
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
from sklearn.experimental import enable_iterative_imputer

from keras.layers import Dense, Dropout
from keras.models import Sequential

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from tqdm import tqdm

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb
from sklearn.metrics import log_loss
from xgboost.sklearn import XGBClassifier
from xgboost.sklearn import XGBRegressor

from Modules import Modules
from Baseline import Baseline

%matplotlib inline

print("OK\n")

OK



In [6]:
app_train_origin = pd.read_csv('./home-credit-default-risk/exports/app_train.csv')
app_test_origin =  pd.read_csv('./home-credit-default-risk/exports/app_test.csv')
bureau_origin = pd.read_csv('./home-credit-default-risk/bureau.csv')
bureau_balance_origin = pd.read_csv('./home-credit-default-risk/bureau_balance.csv')
pos_cash_balance_origin = pd.read_csv('./home-credit-default-risk/POS_CASH_balance.csv')
credit_card_balance_origin = pd.read_csv('./home-credit-default-risk/credit_card_balance.csv')
previous_application_origin = pd.read_csv('./home-credit-default-risk/previous_application.csv')
installments_payments_origin = pd.read_csv('./home-credit-default-risk/installments_payments.csv')

app_train_origin.head()

,LANDAREA_MEDI,NAME_TYPE_SUITE,DAYS_REGISTRATION,DAYS_EMPLOYED,CNT_CHILDREN,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MODE,TARGET,NAME_EDUCATION_TYPE,FLAG_MOBIL,...,DAYS_ID_PUBLISH,DAYS_BIRTH,NAME_CONTRACT_TYPE,CODE_GENDER,AMT_ANNUITY,FLOORSMIN_MODE,LIVINGAPARTMENTS_MODE,YEARS_BEGINEXPLUATATION_MEDI,BASEMENTAREA_MEDI,AMT_CREDIT
0,1.203650,0,0.379837,-637.0,0,0.975354,1.506565,1,0,1,...,0.579154,-1.506880,0,0,-0.088844,0.1250,1.188621,0.973233,-0.726610,-0.478095
1,1.199766,1,1.078697,-1188.0,0,0.985873,1.506565,0,1,1,...,1.790855,0.166821,0,1,0.670508,0.3333,1.556671,0.978441,-0.377455,1.725450
2,-0.827100,0,0.206116,-225.0,0,-0.995749,-0.663503,0,0,1,...,0.306869,0.689509,1,0,-1.666947,NaN,-0.668443,-1.024592,NaN,-1.152888
3,-0.827100,0,-1.375829,-3039.0,0,-0.995749,-0.663503,0,0,1,...,0.369143,0.680114,0,1,0.266951,NaN,-0.668443,-1.024592,NaN,-0.711430
4,-0.827100,0,0.191639,-3038.0,0,-0.995749,-0.663503,0,0,1,...,-0.307263,0.892535,0,0,-0.301776,NaN,-0.668443,-1.024592,NaN,-0.213734


In [7]:
app_train = app_train_origin.copy()
app_test = app_test_origin.copy()
bureau = bureau_origin.copy()
bureau_balance = bureau_balance_origin.copy()
pos_cash_balance = pos_cash_balance_origin.copy()
credit_card_balance = credit_card_balance_origin.copy()
previous_application = previous_application_origin.copy()
installments_payments = installments_payments_origin.copy()

## 共通処理

In [8]:
import xgboost as xgb
from sklearn.metrics import log_loss

def train_xgb_____(train_x
    ,train_y
    ,test_x
    ,params):
    y_preds = []
    models = []
    scores = []
    oof_train = np.zeros((len(train_x),))
    cv = KFold(n_splits=5, shuffle=True, random_state=0)

    num_round = 50

    for fold_id, (train_index, valid_index) in enumerate(cv.split(train_x)):
        X_tr = train_x.loc[train_index, :]
        X_val = train_x.loc[valid_index, :]
        y_tr = train_y[train_index]
        y_val = train_y[valid_index]

        dtrain = xgb.DMatrix(X_tr, label=y_tr)
        dvalid = xgb.DMatrix(X_val, label=y_val)
        dtest = xgb.DMatrix(test_x)

        watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
        model = xgb.train(params, dtrain, num_round, evals=watchlist)

        va_pred = model.predict(dvalid)
        score = log_loss(y_val, va_pred)
        oof_train[valid_index] = va_pred

        y_pred = model.predict(dtest) 

        y_preds.append(y_pred)
        models.append(model)
        scores.append(score)
        
    return y_preds, models, scores

In [9]:
# Plots the disribution of a variable colored by value of the target
def kde_target(var_name, df):
    
    # Calculate the correlation coefficient between the new variable and the target
    corr = df['TARGET'].corr(df[var_name])
    
    # Calculate medians for repaid vs not repaid
    avg_repaid = df.loc[df['TARGET'] == 0, var_name].median()
    avg_not_repaid = df.loc[df['TARGET'] == 1, var_name].median()
    
    plt.figure(figsize = (12, 6))
    
    # Plot the distribution for target == 0 and target == 1
    sns.kdeplot(df.loc[df['TARGET'] == 0, var_name], label = 'TARGET == 0')
    sns.kdeplot(df.loc[df['TARGET'] == 1, var_name], label = 'TARGET == 1')
    
    # label the plot
    plt.xlabel(var_name); plt.ylabel('Density'); plt.title('%s Distribution' % var_name)
    plt.legend();
    
    # print out the correlation
    print('The correlation between %s and the TARGET is %0.4f' % (var_name, corr))
    # Print out average values
    print('Median value for loan that was not repaid = %0.4f' % avg_not_repaid)
    print('Median value for loan that was repaid =     %0.4f' % avg_repaid)

In [10]:
params_base = {
        'booster': 'gbtree'
        ,'objective': 'binary:logistic'
        ,'eta': 0.1
        ,'gamma': 0.0
        ,'alpha': 0.0
        ,'lambda': 1.0
        ,'min_child_weight': 1
        ,'max_depth': 5
        ,'subsample': 0.8
        ,'colsample_bytree': 0.8
        ,'random_state': 17
        ,'gpu_id': 0
        ,'tree_method': 'gpu_hist'
        ,'eval_metric': 'auc'
        ,'colsample_bylevel': 0.3
        ,'num_round': 100000
}


def objective(trial):
    # 探索するパラメータ
    param_tuning = {
            "min_child_weight": trial.suggest_float("min_child_weight", 0.1, 10, log=True)
            ,"max_depth": trial.suggest_int("max_depth", 1, 9, step=1, log=False)
            ,"subsample": trial.suggest_float("subsample", 0.1, 0.95,  step =0.05, log=False)
            ,"colsample_bytree": trial.suggest_float("colsample_bytree", 0.1, 0.95,  step =0.05, log=False)
            ,'colsample_bylevel': trial.suggest_float("colsample_bylevel", 0.1, 0.95,  step =0.05, log=False)
            ,"gamma": trial.suggest_float("gamma", 1e-8, 1.0, log=True)
            # 余裕があれば調整する
            ,"alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True)
            ,"lambda": trial.suggest_float("lambda", 1e-6, 10.0, log=True)
            
    }
    param_tuning.update(params_base)

    # モデル学習・評価:
    list_metrics = []
    cv = list(StratifiedKFold(n_splits=5, shuffle=True, random_state=123).split(x_train, y_train))
    list_fold = [0]
    for nfold in list_fold:
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        x_tr, y_tr = x_train.loc[idx_tr, :], y_train[idx_tr]
        x_va, y_va = x_train.loc[idx_va, :], y_train[idx_va]
        model = xgb.XGBClassifier(**params)
        model.fit(x_tr
                    ,y_tr
                    ,eval_set=[(x_tr, y_tr), (x_va, y_va)]
        )
        y_va_pred = model.predict_proba(x_va)[:,1]
        metric_va = roc_auc_score(y_va, y_va_pred)
        list_metrics.append(metric_va)

    # 評価指数の算出
    metrics = np.mean(list_metrics)

    return metrics

In [11]:
def geometric_mean(x):
    return np.exp(np.log(x[x>0]).mean())

## baselineを確認(ラベルエンコーディング済み)

In [12]:
# app_train
display(app_train['DAYS_EMPLOYED'].value_counts())
print("正の値の割合: {:.4f}".format((app_train['DAYS_EMPLOYED']>0).mean()))
print("正の値の個数: {}".format((app_train['DAYS_EMPLOYED']>0).sum()))

-200.0      156
-224.0      152
-199.0      151
-230.0      151
-212.0      150
           ... 
-13961.0      1
-11827.0      1
-10176.0      1
-9459.0       1
-8694.0       1
Name: DAYS_EMPLOYED, Length: 12573, dtype: int64

正の値の割合: 0.0000
正の値の個数: 0


In [13]:
# app_test
display(app_test['DAYS_EMPLOYED'].value_counts())
print("正の値の割合: {:.4f}".format((app_test['DAYS_EMPLOYED']>0).mean()))
print("正の値の個数: {}".format((app_test['DAYS_EMPLOYED']>0).sum()))

-1119.0     32
-389.0      31
-1240.0     30
-148.0      28
-277.0      27
            ..
-7562.0      1
-6014.0      1
-9748.0      1
-10068.0     1
-6551.0      1
Name: DAYS_EMPLOYED, Length: 7862, dtype: int64

正の値の割合: 0.0000
正の値の個数: 0


In [14]:
params = {
        'booster': 'gbtree'
        ,'objective': 'binary:logistic'
        ,'silent': 1
        ,'random_state': 17
        ,'gpu_id': 0
        ,'tree_method': 'gpu_hist'
        ,'eval_metric': 'auc'
        ,'eta': 0.1
        #,'max_round': 100
        ,'max_depth': 8
        ,'subsample': 0.8
        ,'colsample_state': 71
        ,'colsample_bytree': 1.0
        ,'colsample_bylevel': 0.3
        ,'subsumple': 0.9
        ,'gamma': 0
        ,'lambda': 1
        ,'alpha': 0
        ,'min_child_weight': 1
}

In [15]:
x_train = app_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = app_train['TARGET']
id_train = app_train[['SK_ID_CURR']]

In [16]:
baseline = Baseline('SK_ID_CURR')

train_oof, imp, metrics = baseline.train_xgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 82) (61503, 82)
[21:37:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_state", "silent", "subsumple" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-auc:0.72702	validation_1-auc:0.71274
[1]	validation_0-auc:0.73630	validation_1-auc:0.72259
[2]	validation_0-auc:0.74283	validation_1-auc:0.72617
[3]	validation_0-auc:0.74527	validation_1-auc:0.72786
[4]	validation_0-auc:0.74975	validation_1-auc:0.73050
[5]	validation_0-auc:0.75273	validation_1-auc:0.73337
[6]	validation_0-auc:0.75695	validation_1-auc:0.73543
[7]	validation_0-auc:0.75897	validation_1-auc:0.73719
[8]	validation_0-auc:0.76050	validation_1-auc:0.7375

[31]	validation_0-auc:0.79064	validation_1-auc:0.74467
[32]	validation_0-auc:0.79213	validation_1-auc:0.74491
[33]	validation_0-auc:0.79349	validation_1-auc:0.74503
[34]	validation_0-auc:0.79489	validation_1-auc:0.74554
[35]	validation_0-auc:0.79603	validation_1-auc:0.74586
[36]	validation_0-auc:0.79756	validation_1-auc:0.74664
[37]	validation_0-auc:0.79840	validation_1-auc:0.74688
[38]	validation_0-auc:0.79909	validation_1-auc:0.74733
[39]	validation_0-auc:0.80052	validation_1-auc:0.74781
[40]	validation_0-auc:0.80176	validation_1-auc:0.74804
[41]	validation_0-auc:0.80305	validation_1-auc:0.74819
[42]	validation_0-auc:0.80429	validation_1-auc:0.74851
[43]	validation_0-auc:0.80526	validation_1-auc:0.74878
[44]	validation_0-auc:0.80626	validation_1-auc:0.74913
[45]	validation_0-auc:0.80772	validation_1-auc:0.74916
[46]	validation_0-auc:0.80894	validation_1-auc:0.74939
[47]	validation_0-auc:0.80984	validation_1-auc:0.74971
[48]	validation_0-auc:0.81072	validation_1-auc:0.74994
[49]	valid

[71]	validation_0-auc:0.82939	validation_1-auc:0.75783
[72]	validation_0-auc:0.83015	validation_1-auc:0.75784
[73]	validation_0-auc:0.83095	validation_1-auc:0.75784
[74]	validation_0-auc:0.83198	validation_1-auc:0.75778
[75]	validation_0-auc:0.83302	validation_1-auc:0.75801
[76]	validation_0-auc:0.83425	validation_1-auc:0.75787
[77]	validation_0-auc:0.83493	validation_1-auc:0.75787
[78]	validation_0-auc:0.83521	validation_1-auc:0.75790
[79]	validation_0-auc:0.83601	validation_1-auc:0.75791
[80]	validation_0-auc:0.83710	validation_1-auc:0.75804
[81]	validation_0-auc:0.83788	validation_1-auc:0.75815
[82]	validation_0-auc:0.83860	validation_1-auc:0.75811
[83]	validation_0-auc:0.83910	validation_1-auc:0.75802
[84]	validation_0-auc:0.84008	validation_1-auc:0.75803
[85]	validation_0-auc:0.84089	validation_1-auc:0.75806
[86]	validation_0-auc:0.84182	validation_1-auc:0.75811
[87]	validation_0-auc:0.84281	validation_1-auc:0.75813
[88]	validation_0-auc:0.84367	validation_1-auc:0.75828
[89]	valid

[1]	validation_0-auc:0.73921	validation_1-auc:0.71827
[2]	validation_0-auc:0.74506	validation_1-auc:0.72053
[3]	validation_0-auc:0.74777	validation_1-auc:0.72197
[4]	validation_0-auc:0.75110	validation_1-auc:0.72543
[5]	validation_0-auc:0.75407	validation_1-auc:0.72751
[6]	validation_0-auc:0.75792	validation_1-auc:0.72952
[7]	validation_0-auc:0.75985	validation_1-auc:0.73036
[8]	validation_0-auc:0.76167	validation_1-auc:0.73119
[9]	validation_0-auc:0.76361	validation_1-auc:0.73091
[10]	validation_0-auc:0.76619	validation_1-auc:0.73165
[11]	validation_0-auc:0.76755	validation_1-auc:0.73255
[12]	validation_0-auc:0.76829	validation_1-auc:0.73295
[13]	validation_0-auc:0.76910	validation_1-auc:0.73331
[14]	validation_0-auc:0.77051	validation_1-auc:0.73382
[15]	validation_0-auc:0.77139	validation_1-auc:0.73382
[16]	validation_0-auc:0.77276	validation_1-auc:0.73420
[17]	validation_0-auc:0.77347	validation_1-auc:0.73455
[18]	validation_0-auc:0.77455	validation_1-auc:0.73521
[19]	validation_0-a

## feature enginniring

In [17]:
# Kaggleから提供されているCSVを読み込み
app_train_origin_k =  pd.read_csv('./home-credit-default-risk/application_train.csv')
app_test_origin_k =  pd.read_csv('./home-credit-default-risk/application_test.csv')
app_train_origin.head()

,LANDAREA_MEDI,NAME_TYPE_SUITE,DAYS_REGISTRATION,DAYS_EMPLOYED,CNT_CHILDREN,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MODE,TARGET,NAME_EDUCATION_TYPE,FLAG_MOBIL,...,DAYS_ID_PUBLISH,DAYS_BIRTH,NAME_CONTRACT_TYPE,CODE_GENDER,AMT_ANNUITY,FLOORSMIN_MODE,LIVINGAPARTMENTS_MODE,YEARS_BEGINEXPLUATATION_MEDI,BASEMENTAREA_MEDI,AMT_CREDIT
0,1.203650,0,0.379837,-637.0,0,0.975354,1.506565,1,0,1,...,0.579154,-1.506880,0,0,-0.088844,0.1250,1.188621,0.973233,-0.726610,-0.478095
1,1.199766,1,1.078697,-1188.0,0,0.985873,1.506565,0,1,1,...,1.790855,0.166821,0,1,0.670508,0.3333,1.556671,0.978441,-0.377455,1.725450
2,-0.827100,0,0.206116,-225.0,0,-0.995749,-0.663503,0,0,1,...,0.306869,0.689509,1,0,-1.666947,NaN,-0.668443,-1.024592,NaN,-1.152888
3,-0.827100,0,-1.375829,-3039.0,0,-0.995749,-0.663503,0,0,1,...,0.369143,0.680114,0,1,0.266951,NaN,-0.668443,-1.024592,NaN,-0.711430
4,-0.827100,0,0.191639,-3038.0,0,-0.995749,-0.663503,0,0,1,...,-0.307263,0.892535,0,0,-0.301776,NaN,-0.668443,-1.024592,NaN,-0.213734


### one-hot-encoding

In [18]:
app_train = app_train_origin_k.copy()
app_test = app_test_origin_k.copy()
app_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
# one-hotencoding
encoding_columns = []

for col in app_test.columns:
    if app_test[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

#encoding_columns

NAME_CONTRACT_TYPE
CODE_GENDER
FLAG_OWN_CAR
FLAG_OWN_REALTY
NAME_TYPE_SUITE
NAME_INCOME_TYPE
NAME_EDUCATION_TYPE
NAME_FAMILY_STATUS
NAME_HOUSING_TYPE
OCCUPATION_TYPE
WEEKDAY_APPR_PROCESS_START
ORGANIZATION_TYPE
FONDKAPREMONT_MODE
HOUSETYPE_MODE
WALLSMATERIAL_MODE
EMERGENCYSTATE_MODE


In [20]:
### CODE_GENDER
app_train['CODE_GENDER'].replace(['F', 'M', 'XNA'], [0, 1, 2], inplace=True)
app_test['CODE_GENDER'].replace(['F', 'M', 'XNA'], [0, 1, 2], inplace=True)
encoding_columns.remove('CODE_GENDER')
### NAME_INCOME_TYPE
app_train['NAME_INCOME_TYPE'].replace(['Working','Commercial associate','Pensioner','State servant','Unemployed','Student','Businessman','Maternity leave'], [0, 1, 2, 3, 4, 5, 6, 7], inplace=True)
app_test['NAME_INCOME_TYPE'].replace(['Working','Commercial associate','Pensioner','State servant','Unemployed','Student','Businessman','Maternity leave'], [0, 1, 2, 3, 4, 5, 6, 7], inplace=True)
encoding_columns.remove('NAME_INCOME_TYPE')
### NAME_FAMILY_STATUS
app_train['NAME_FAMILY_STATUS'].replace(['Married','Single / not married','Civil marriage','Separated','Widow','Unknown'], [0, 1, 2, 3, 4, 5], inplace=True)
app_test['NAME_FAMILY_STATUS'].replace(['Married','Single / not married','Civil marriage','Separated','Widow','Unknown'], [0, 1, 2, 3, 4, 5], inplace=True)
encoding_columns.remove('NAME_FAMILY_STATUS')

In [21]:
for col in encoding_columns:
    app_train = pd.get_dummies(app_train, columns=[col], dummy_na=True)
    app_test = pd.get_dummies(app_test, columns=[col], dummy_na=True)
    
app_train.head()

,SK_ID_CURR,TARGET,CODE_GENDER,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_INCOME_TYPE,NAME_FAMILY_STATUS,...,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_nan,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,EMERGENCYSTATE_MODE_nan
0,100002,1,1,0,202500.0,406597.5,24700.5,351000.0,0,1,...,0,0,0,0,1,0,0,1,0,0
1,100003,0,0,0,270000.0,1293502.5,35698.5,1129500.0,3,0,...,0,0,0,0,0,0,0,1,0,0
2,100004,0,1,0,67500.0,135000.0,6750.0,135000.0,0,1,...,0,0,0,0,0,0,1,0,0,1
3,100006,0,0,0,135000.0,312682.5,29686.5,297000.0,0,2,...,0,0,0,0,0,0,1,0,0,1
4,100007,0,1,0,121500.0,513000.0,21865.5,513000.0,0,1,...,0,0,0,0,0,0,1,0,0,1


In [22]:
len(app_train.columns)

245

In [23]:
app_train.columns

Index(['SK_ID_CURR', 'TARGET', 'CODE_GENDER', 'CNT_CHILDREN',
       'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE',
       'NAME_INCOME_TYPE', 'NAME_FAMILY_STATUS',
       ...
       'WALLSMATERIAL_MODE_Mixed', 'WALLSMATERIAL_MODE_Monolithic',
       'WALLSMATERIAL_MODE_Others', 'WALLSMATERIAL_MODE_Panel',
       'WALLSMATERIAL_MODE_Stone, brick', 'WALLSMATERIAL_MODE_Wooden',
       'WALLSMATERIAL_MODE_nan', 'EMERGENCYSTATE_MODE_No',
       'EMERGENCYSTATE_MODE_Yes', 'EMERGENCYSTATE_MODE_nan'],
      dtype='object', length=245)

In [24]:
app_test.head()

,SK_ID_CURR,CODE_GENDER,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_INCOME_TYPE,NAME_FAMILY_STATUS,REGION_POPULATION_RELATIVE,...,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,WALLSMATERIAL_MODE_nan,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,EMERGENCYSTATE_MODE_nan
0,100001,0,0,135000.0,568800.0,20560.5,450000.0,0,0,0.018850,...,0,0,0,0,1,0,0,1,0,0
1,100005,1,0,99000.0,222768.0,17370.0,180000.0,0,0,0.035792,...,0,0,0,0,0,0,1,0,0,1
2,100013,1,0,202500.0,663264.0,69777.0,630000.0,0,0,0.019101,...,0,0,0,0,0,0,1,0,0,1
3,100028,0,2,315000.0,1575000.0,49018.5,1575000.0,0,0,0.026392,...,0,0,0,1,0,0,0,1,0,0
4,100038,1,1,180000.0,625500.0,32067.0,625500.0,0,0,0.010032,...,0,0,0,0,0,0,1,0,0,1


In [25]:
len(app_test.columns)

244

In [26]:
app_test.columns

Index(['SK_ID_CURR', 'CODE_GENDER', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'NAME_INCOME_TYPE',
       'NAME_FAMILY_STATUS', 'REGION_POPULATION_RELATIVE',
       ...
       'WALLSMATERIAL_MODE_Mixed', 'WALLSMATERIAL_MODE_Monolithic',
       'WALLSMATERIAL_MODE_Others', 'WALLSMATERIAL_MODE_Panel',
       'WALLSMATERIAL_MODE_Stone, brick', 'WALLSMATERIAL_MODE_Wooden',
       'WALLSMATERIAL_MODE_nan', 'EMERGENCYSTATE_MODE_No',
       'EMERGENCYSTATE_MODE_Yes', 'EMERGENCYSTATE_MODE_nan'],
      dtype='object', length=244)

In [27]:
x_train = app_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = app_train['TARGET']
id_train = app_train[['SK_ID_CURR']]

In [28]:
baseline = Baseline('SK_ID_CURR')

train_oof, imp, metrics = baseline.train_xgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 243) (61503, 243)
[21:38:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_state", "silent", "subsumple" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-auc:0.72309	validation_1-auc:0.70909
[1]	validation_0-auc:0.74029	validation_1-auc:0.72529
[2]	validation_0-auc:0.74802	validation_1-auc:0.72946
[3]	validation_0-auc:0.75192	validation_1-auc:0.73179
[4]	validation_0-auc:0.75493	validation_1-auc:0.73345
[5]	validation_0-auc:0.75701	validation_1-auc:0.73437
[6]	validation_0-auc:0.75889	validation_1-auc:0.73492
[7]	validation_0-auc:0.75993	validation_1-auc:0.73479
[8]	validation_0-auc:0.76224	validation_1-auc:0.73

[30]	validation_0-auc:0.79255	validation_1-auc:0.74575
[31]	validation_0-auc:0.79394	validation_1-auc:0.74625
[32]	validation_0-auc:0.79572	validation_1-auc:0.74670
[33]	validation_0-auc:0.79711	validation_1-auc:0.74731
[34]	validation_0-auc:0.79826	validation_1-auc:0.74797
[35]	validation_0-auc:0.79958	validation_1-auc:0.74820
[36]	validation_0-auc:0.80072	validation_1-auc:0.74859
[37]	validation_0-auc:0.80143	validation_1-auc:0.74898
[38]	validation_0-auc:0.80218	validation_1-auc:0.74942
[39]	validation_0-auc:0.80351	validation_1-auc:0.74935
[40]	validation_0-auc:0.80452	validation_1-auc:0.74972
[41]	validation_0-auc:0.80588	validation_1-auc:0.75009
[42]	validation_0-auc:0.80718	validation_1-auc:0.75018
[43]	validation_0-auc:0.80819	validation_1-auc:0.75053
[44]	validation_0-auc:0.80949	validation_1-auc:0.75065
[45]	validation_0-auc:0.81121	validation_1-auc:0.75131
[46]	validation_0-auc:0.81234	validation_1-auc:0.75185
[47]	validation_0-auc:0.81303	validation_1-auc:0.75189
[48]	valid

[70]	validation_0-auc:0.83513	validation_1-auc:0.75742
[71]	validation_0-auc:0.83562	validation_1-auc:0.75740
[72]	validation_0-auc:0.83611	validation_1-auc:0.75753
[73]	validation_0-auc:0.83716	validation_1-auc:0.75750
[74]	validation_0-auc:0.83777	validation_1-auc:0.75756
[75]	validation_0-auc:0.83819	validation_1-auc:0.75761
[76]	validation_0-auc:0.83887	validation_1-auc:0.75766
[77]	validation_0-auc:0.83943	validation_1-auc:0.75764
[78]	validation_0-auc:0.83989	validation_1-auc:0.75776
[79]	validation_0-auc:0.84102	validation_1-auc:0.75767
[80]	validation_0-auc:0.84206	validation_1-auc:0.75750
[81]	validation_0-auc:0.84294	validation_1-auc:0.75744
[82]	validation_0-auc:0.84417	validation_1-auc:0.75751
[83]	validation_0-auc:0.84489	validation_1-auc:0.75761
[84]	validation_0-auc:0.84560	validation_1-auc:0.75759
[85]	validation_0-auc:0.84615	validation_1-auc:0.75769
[86]	validation_0-auc:0.84687	validation_1-auc:0.75773
[87]	validation_0-auc:0.84776	validation_1-auc:0.75754
[88]	valid

[0]	validation_0-auc:0.72487	validation_1-auc:0.70418
[1]	validation_0-auc:0.74190	validation_1-auc:0.72016
[2]	validation_0-auc:0.74813	validation_1-auc:0.72426
[3]	validation_0-auc:0.75308	validation_1-auc:0.72850
[4]	validation_0-auc:0.75546	validation_1-auc:0.72879
[5]	validation_0-auc:0.75698	validation_1-auc:0.72938
[6]	validation_0-auc:0.75835	validation_1-auc:0.73058
[7]	validation_0-auc:0.75985	validation_1-auc:0.73082
[8]	validation_0-auc:0.76242	validation_1-auc:0.73236
[9]	validation_0-auc:0.76509	validation_1-auc:0.73332
[10]	validation_0-auc:0.76648	validation_1-auc:0.73365
[11]	validation_0-auc:0.76780	validation_1-auc:0.73377
[12]	validation_0-auc:0.76961	validation_1-auc:0.73446
[13]	validation_0-auc:0.77170	validation_1-auc:0.73547
[14]	validation_0-auc:0.77318	validation_1-auc:0.73588
[15]	validation_0-auc:0.77419	validation_1-auc:0.73566
[16]	validation_0-auc:0.77483	validation_1-auc:0.73581
[17]	validation_0-auc:0.77650	validation_1-auc:0.73630
[18]	validation_0-au

### 特徴量作成①

In [29]:
app_train['DAYS_EMPLOYED'] = app_train_origin['DAYS_EMPLOYED'].copy()
app_test['DAYS_EMPLOYED'] = app_test_origin['DAYS_EMPLOYED'].copy()

In [30]:
display(app_train['DAYS_EMPLOYED'].value_counts())
print("正の値の割合: {:.4f}".format((app_train['DAYS_EMPLOYED']>0).mean()))
print("正の値の個数: {}".format((app_train['DAYS_EMPLOYED']>0).sum()))

-200.0      156
-224.0      152
-199.0      151
-230.0      151
-212.0      150
           ... 
-13961.0      1
-11827.0      1
-10176.0      1
-9459.0       1
-8694.0       1
Name: DAYS_EMPLOYED, Length: 12573, dtype: int64

正の値の割合: 0.0000
正の値の個数: 0


In [31]:
display(app_test['DAYS_EMPLOYED'].value_counts())
print("正の値の割合: {:.4f}".format((app_test['DAYS_EMPLOYED']>0).mean()))
print("正の値の個数: {}".format((app_test['DAYS_EMPLOYED']>0).sum()))

-1119.0     32
-389.0      31
-1240.0     30
-148.0      28
-277.0      27
            ..
-7562.0      1
-6014.0      1
-9748.0      1
-10068.0     1
-6551.0      1
Name: DAYS_EMPLOYED, Length: 7862, dtype: int64

正の値の割合: 0.0000
正の値の個数: 0


In [32]:
docs = [_f for _f in app_train.columns if 'FLAG_DOC' in _f]
live = [_f for _f in app_train.columns if ('FLAG_' in _f) & ('FLAG_DOC' not in _f) & ('_FLAG_' not in _f)]
app_train['ORGANIZATION_TYPE'] = app_train_origin['ORGANIZATION_TYPE'].copy()

# nullに置き換え
app_train['DAYS_EMPLOYED'] = app_train['DAYS_EMPLOYED'].replace(365243, np.nan)

inc_by_org = app_train[['AMT_INCOME_TOTAL', 'ORGANIZATION_TYPE']].groupby('ORGANIZATION_TYPE').median()['AMT_INCOME_TOTAL']

# 特徴量1: 総所得金額を世帯人数で除した値
app_train['INCOME_div_PERSON'] = app_train['AMT_INCOME_TOTAL'] / app_train['CNT_FAM_MEMBERS']

# 特徴量2: 総所得額を就労期間で除した値
app_train['INCOME_div_EMPLOYED'] = app_train['AMT_INCOME_TOTAL'] / app_train['DAYS_EMPLOYED']

#
app_train['NEW_CREDIT_TO_ANNUITY_RATIO'] = app_train['AMT_CREDIT'] / app_train['AMT_ANNUITY']

app_train['NEW_CREDIT_TO_GOODS_RATIO'] = app_train['AMT_CREDIT'] / app_train['AMT_GOODS_PRICE']

app_train['NEW_DOC_IND_KURT'] = app_train[docs].kurtosis(axis=1)

app_train['NEW_LIVE_IND_SUM'] = app_train[live].sum(axis=1)

app_train['NEW_INC_PER_CHLD'] = app_train['AMT_INCOME_TOTAL'] / (1 + app_train['CNT_CHILDREN'])

app_train['NEW_INC_BY_ORG'] = app_train['ORGANIZATION_TYPE'].map(inc_by_org)

app_train['NEW_EMPLOY_TO_BIRTH_RATIO'] = app_train['DAYS_EMPLOYED'] / app_train['DAYS_BIRTH']

app_train['NEW_EMPLOY_TO_BIRTH-18_RATIO'] = app_train['DAYS_EMPLOYED'] / (app_train['DAYS_BIRTH'] + 18*365)

app_train['NEW_BIRTH_TO_EMPLOY_RATIO'] = app_train['DAYS_BIRTH'] / (1 + app_train['DAYS_EMPLOYED'])

app_train['NEW_INCOME_TO_ANNUITY_RATIO'] = app_train['AMT_INCOME_TOTAL'] / (1 + app_train['AMT_ANNUITY'])

app_train['NEW_ANNUITY_TO_INCOME_RATIO'] = app_train['AMT_ANNUITY'] / (1 + app_train['AMT_INCOME_TOTAL'])


# 特徴量3: 外部スコアの平均
app_train['NEW_EXT_SOURCES_MEDIAN'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].median(axis=1, skipna=True)

app_train['EXT_SOURCE_mean'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
    
app_train['EXT_SOURCE_max'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].max(axis=1)

app_train['EXT_SOURCE_min'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].min(axis=1)

app_train['EXT_SOURCE_std'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)

app_train['EXT_SOURCE_count'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].notnull().sum(axis=1)

app_train['NEW_EXT_SOURCES_PROD'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].prod(axis=1, skipna=True, min_count=1)

app_train['NEW_EXT_SOURCES_MAD'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mad(axis=1, skipna=True)

app_train['NEW_EXT_SOURCES_GEO'] = app_train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].apply(geometric_mean, axis=1)

# 特徴量4: 就労期間を年齢で除した値
app_train['DAYS_EMPLOYED_div_BERTH'] = app_train['DAYS_EMPLOYED'] / app_train['DAYS_BIRTH']

# 特徴量5: 年金支払金額を所得金額で除した値
app_train['ANNUITY_div_INCOME'] = app_train['AMT_ANNUITY'] / app_train['AMT_INCOME_TOTAL']

# 特徴量6: 年金支払額を借入金で除した値
app_train['ANNUITY_div_CREDIT'] = app_train['AMT_ANNUITY'] / app_train['AMT_CREDIT']

# 
app_train['credit_goods_price_ratio'] = app_train['AMT_CREDIT'] / app_train['AMT_GOODS_PRICE']
 
app_train['credit_downpayment'] = app_train['AMT_GOODS_PRICE'] - app_train['AMT_CREDIT']

app_train['USER_AGE'] = abs(app_train['DAYS_BIRTH'])/364.25

app_train['NEW_CAR_TO_BIRTH_RATIO'] = app_train['OWN_CAR_AGE'] / app_train['DAYS_BIRTH']

app_train['NEW_CAR_TO_EMPLOY_RATIO'] = app_train['OWN_CAR_AGE'] / app_train['DAYS_EMPLOYED']

app_train['NEW_PHONE_TO_BIRTH_RATIO'] = app_train['DAYS_LAST_PHONE_CHANGE'] / app_train['DAYS_BIRTH']

app_train['NEW_PHONE_TO_EMPLOYED_RATIO'] = app_train['DAYS_LAST_PHONE_CHANGE'] / app_train['DAYS_EMPLOYED']

app_train['NEW_CREDIT_TO_INCOME_RATIO'] = app_train['AMT_CREDIT'] / app_train['AMT_INCOME_TOTAL']

app_train['NEW_INCOME_CREDIT_PERC'] = app_train['AMT_INCOME_TOTAL'] / app_train['AMT_CREDIT']

app_train['NEW_INCOME_PER_PERSON'] = app_train['AMT_INCOME_TOTAL'] / app_train['CNT_FAM_MEMBERS']


app_train.drop('ORGANIZATION_TYPE', axis=1, inplace=True)

len(app_train.columns)

,SK_ID_CURR,TARGET,CODE_GENDER,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_INCOME_TYPE,NAME_FAMILY_STATUS,...,PHONE_TO_EMPLOY_RATIO,app EXT_SOURCE_1_times_EXT_SOURCE_2,app EXT_SOURCE_1_times_EXT_SOURCE_3,app EXT_SOURCE_2_times_EXT_SOURCE_3,app EXT_SOURCE_1_times_DAYS_EMPLOYED,app EXT_SOURCE_2_times_DAYS_EMPLOYED,app EXT_SOURCE_3_times_DAYS_EMPLOYED,app EXT_SOURCE_1_div_DAYS_BIRTH,app EXT_SOURCE_2_div_DAYS_BIRTH,app EXT_SOURCE_3_div_DAYS_BIRTH
0,100002,1,1,0,202500.0,406597.5,24700.5,351000.0,0,1,...,1.780220,0.021834,0.011573,0.036649,-52.894548,-167.498254,-88.782372,-0.000009,-0.000028,-0.000015
1,100003,0,0,0,270000.0,1293502.5,35698.5,1129500.0,3,0,...,0.696970,0.193685,NaN,NaN,-369.785566,-739.227981,NaN,-0.000019,-0.000037,NaN
2,100004,0,1,0,67500.0,135000.0,6750.0,135000.0,0,1,...,3.622222,NaN,NaN,0.405575,NaN,-125.080219,-164.152505,NaN,-0.000029,-0.000038
3,100006,0,0,0,135000.0,312682.5,29686.5,297000.0,0,2,...,0.203027,NaN,NaN,NaN,NaN,-1976.692297,NaN,NaN,-0.000034,NaN
4,100007,0,1,0,121500.0,513000.0,21865.5,513000.0,0,1,...,0.364055,NaN,NaN,NaN,NaN,-980.478916,NaN,NaN,-0.000016,NaN


In [33]:
docs = [_f for _f in app_test.columns if 'FLAG_DOC' in _f]
live = [_f for _f in app_test.columns if ('FLAG_' in _f) & ('FLAG_DOC' not in _f) & ('_FLAG_' not in _f)]
app_test['ORGANIZATION_TYPE'] = app_test_origin['ORGANIZATION_TYPE'].copy()

# nullに置き換え
app_test['DAYS_EMPLOYED'] = app_test['DAYS_EMPLOYED'].replace(365243, np.nan)

inc_by_org = app_test[['AMT_INCOME_TOTAL', 'ORGANIZATION_TYPE']].groupby('ORGANIZATION_TYPE').median()['AMT_INCOME_TOTAL']

# 特徴量1: 総所得金額を世帯人数で除した値
app_test['INCOME_div_PERSON'] = app_test['AMT_INCOME_TOTAL'] / app_test['CNT_FAM_MEMBERS']

# 特徴量2: 総所得額を就労期間で除した値
app_test['INCOME_div_EMPLOYED'] = app_test['AMT_INCOME_TOTAL'] / app_test['DAYS_EMPLOYED']

#
app_test['NEW_CREDIT_TO_ANNUITY_RATIO'] = app_test['AMT_CREDIT'] / app_test['AMT_ANNUITY']

app_test['NEW_CREDIT_TO_GOODS_RATIO'] = app_test['AMT_CREDIT'] / app_test['AMT_GOODS_PRICE']

app_test['NEW_DOC_IND_KURT'] = app_test[docs].kurtosis(axis=1)

app_test['NEW_LIVE_IND_SUM'] = app_test[live].sum(axis=1)

app_test['NEW_INC_PER_CHLD'] = app_test['AMT_INCOME_TOTAL'] / (1 + app_test['CNT_CHILDREN'])

app_test['NEW_INC_BY_ORG'] = app_test['ORGANIZATION_TYPE'].map(inc_by_org)

app_test['NEW_EMPLOY_TO_BIRTH_RATIO'] = app_test['DAYS_EMPLOYED'] / app_test['DAYS_BIRTH']

app_test['NEW_EMPLOY_TO_BIRTH-18_RATIO'] = app_test['DAYS_EMPLOYED'] / (app_test['DAYS_BIRTH'] + 18*365)

app_test['NEW_BIRTH_TO_EMPLOY_RATIO'] = app_test['DAYS_BIRTH'] / (1 + app_test['DAYS_EMPLOYED'])

app_test['NEW_INCOME_TO_ANNUITY_RATIO'] = app_test['AMT_INCOME_TOTAL'] / (1 + app_test['AMT_ANNUITY'])

app_test['NEW_ANNUITY_TO_INCOME_RATIO'] = app_test['AMT_ANNUITY'] / (1 + app_test['AMT_INCOME_TOTAL'])


# 特徴量3: 外部スコアの平均
app_test['NEW_EXT_SOURCES_MEDIAN'] = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].median(axis=1, skipna=True)

app_test['EXT_SOURCE_mean'] = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
    
app_test['EXT_SOURCE_max'] = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].max(axis=1)

app_test['EXT_SOURCE_min'] = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].min(axis=1)

app_test['EXT_SOURCE_std'] = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)

app_test['EXT_SOURCE_count'] = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].notnull().sum(axis=1)

app_test['NEW_EXT_SOURCES_PROD'] = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].prod(axis=1, skipna=True, min_count=1)

app_test['NEW_EXT_SOURCES_MAD'] = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mad(axis=1, skipna=True)

app_test['NEW_EXT_SOURCES_GEO'] = app_test[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].apply(geometric_mean, axis=1)

# 特徴量4: 就労期間を年齢で除した値
app_test['DAYS_EMPLOYED_div_BERTH'] = app_test['DAYS_EMPLOYED'] / app_test['DAYS_BIRTH']

# 特徴量5: 年金支払金額を所得金額で除した値
app_test['ANNUITY_div_INCOME'] = app_test['AMT_ANNUITY'] / app_test['AMT_INCOME_TOTAL']

# 特徴量6: 年金支払額を借入金で除した値
app_test['ANNUITY_div_CREDIT'] = app_test['AMT_ANNUITY'] / app_test['AMT_CREDIT']

# 
app_test['credit_goods_price_ratio'] = app_test['AMT_CREDIT'] / app_test['AMT_GOODS_PRICE']
 
app_test['credit_downpayment'] = app_test['AMT_GOODS_PRICE'] - app_test['AMT_CREDIT']

app_test['USER_AGE'] = abs(app_test['DAYS_BIRTH'])/364.25

app_test['NEW_CAR_TO_BIRTH_RATIO'] = app_test['OWN_CAR_AGE'] / app_test['DAYS_BIRTH']

app_test['NEW_CAR_TO_EMPLOY_RATIO'] = app_test['OWN_CAR_AGE'] / app_test['DAYS_EMPLOYED']

app_test['NEW_PHONE_TO_BIRTH_RATIO'] = app_test['DAYS_LAST_PHONE_CHANGE'] / app_test['DAYS_BIRTH']

app_test['NEW_PHONE_TO_EMPLOYED_RATIO'] = app_test['DAYS_LAST_PHONE_CHANGE'] / app_test['DAYS_EMPLOYED']

app_test['NEW_CREDIT_TO_INCOME_RATIO'] = app_test['AMT_CREDIT'] / app_test['AMT_INCOME_TOTAL']

app_test['NEW_INCOME_CREDIT_PERC'] = app_test['AMT_INCOME_TOTAL'] / app_test['AMT_CREDIT']

app_test['NEW_INCOME_PER_PERSON'] = app_test['AMT_INCOME_TOTAL'] / app_test['CNT_FAM_MEMBERS']


app_test.drop('ORGANIZATION_TYPE', axis=1, inplace=True)

len(app_test.columns)

,SK_ID_CURR,CODE_GENDER,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_INCOME_TYPE,NAME_FAMILY_STATUS,REGION_POPULATION_RELATIVE,...,PHONE_TO_EMPLOY_RATIO,app EXT_SOURCE_1_times_EXT_SOURCE_2,app EXT_SOURCE_1_times_EXT_SOURCE_3,app EXT_SOURCE_2_times_EXT_SOURCE_3,app EXT_SOURCE_1_times_DAYS_EMPLOYED,app EXT_SOURCE_2_times_DAYS_EMPLOYED,app EXT_SOURCE_3_times_DAYS_EMPLOYED,app EXT_SOURCE_1_div_DAYS_BIRTH,app EXT_SOURCE_2_div_DAYS_BIRTH,app EXT_SOURCE_3_div_DAYS_BIRTH
0,100001,0,0,135000.0,568800.0,20560.5,450000.0,0,0,0.018850,...,0.747102,0.594305,0.120057,0.125965,-1752.839149,-1839.104984,-371.521010,-0.000039,-0.000041,-0.000008
1,100005,1,0,99000.0,222768.0,17370.0,180000.0,0,0,0.035792,...,-0.000000,0.164783,0.244619,0.126276,-2524.941212,-1303.408573,-1934.905690,-0.000031,-0.000016,-0.000024
2,100013,1,0,202500.0,663264.0,69777.0,630000.0,0,0,0.019101,...,0.192014,NaN,NaN,0.427564,NaN,-3119.649689,-2723.799341,NaN,-0.000035,-0.000030
3,100028,0,2,315000.0,1575000.0,49018.5,1575000.0,0,0,0.026392,...,0.967310,0.267955,0.322119,0.312281,-981.019602,-951.057432,-1143.306119,-0.000038,-0.000036,-0.000044
4,100038,1,1,180000.0,625500.0,32067.0,625500.0,0,0,0.010032,...,0.374715,0.086051,NaN,NaN,-442.899678,-932.680861,NaN,-0.000016,-0.000033,NaN


In [34]:
# infをnanへ置換
app_train = app_train.replace([np.inf, -np.inf], np.nan)
app_test = app_test.replace([np.inf, -np.inf], np.nan)

In [35]:
x_train = app_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = app_train['TARGET']
id_train = app_train[['SK_ID_CURR']]

In [36]:
baseline = Baseline('SK_ID_CURR')

train_oof, imp, metrics = baseline.train_xgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 292) (61503, 292)
[21:40:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_state", "silent", "subsumple" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-auc:0.73703	validation_1-auc:0.72117
[1]	validation_0-auc:0.74882	validation_1-auc:0.73083
[2]	validation_0-auc:0.75435	validation_1-auc:0.73360
[3]	validation_0-auc:0.75814	validation_1-auc:0.73617
[4]	validation_0-auc:0.76141	validation_1-auc:0.73728
[5]	validation_0-auc:0.76393	validation_1-auc:0.73830
[6]	validation_0-auc:0.76627	validation_1-auc:0.73818
[7]	validation_0-auc:0.76878	validation_1-auc:0.73799
[8]	validation_0-auc:0.77079	validation_1-auc:0.73

[30]	validation_0-auc:0.80612	validation_1-auc:0.74988
[31]	validation_0-auc:0.80744	validation_1-auc:0.75048
[32]	validation_0-auc:0.80889	validation_1-auc:0.75097
[33]	validation_0-auc:0.81031	validation_1-auc:0.75155
[34]	validation_0-auc:0.81189	validation_1-auc:0.75200
[35]	validation_0-auc:0.81347	validation_1-auc:0.75270
[36]	validation_0-auc:0.81490	validation_1-auc:0.75323
[37]	validation_0-auc:0.81677	validation_1-auc:0.75386
[38]	validation_0-auc:0.81831	validation_1-auc:0.75418
[39]	validation_0-auc:0.81985	validation_1-auc:0.75459
[40]	validation_0-auc:0.82181	validation_1-auc:0.75512
[41]	validation_0-auc:0.82337	validation_1-auc:0.75561
[42]	validation_0-auc:0.82458	validation_1-auc:0.75620
[43]	validation_0-auc:0.82610	validation_1-auc:0.75640
[44]	validation_0-auc:0.82776	validation_1-auc:0.75693
[45]	validation_0-auc:0.82925	validation_1-auc:0.75741
[46]	validation_0-auc:0.83083	validation_1-auc:0.75773
[47]	validation_0-auc:0.83153	validation_1-auc:0.75796
[48]	valid

[70]	validation_0-auc:0.85637	validation_1-auc:0.76506
[71]	validation_0-auc:0.85698	validation_1-auc:0.76509
[72]	validation_0-auc:0.85755	validation_1-auc:0.76515
[73]	validation_0-auc:0.85813	validation_1-auc:0.76525
[74]	validation_0-auc:0.85837	validation_1-auc:0.76535
[75]	validation_0-auc:0.85910	validation_1-auc:0.76554
[76]	validation_0-auc:0.85933	validation_1-auc:0.76558
[77]	validation_0-auc:0.86004	validation_1-auc:0.76551
[78]	validation_0-auc:0.86087	validation_1-auc:0.76553
[79]	validation_0-auc:0.86130	validation_1-auc:0.76556
[80]	validation_0-auc:0.86183	validation_1-auc:0.76551
[81]	validation_0-auc:0.86275	validation_1-auc:0.76556
[82]	validation_0-auc:0.86360	validation_1-auc:0.76550
[83]	validation_0-auc:0.86477	validation_1-auc:0.76547
[84]	validation_0-auc:0.86559	validation_1-auc:0.76530
[85]	validation_0-auc:0.86695	validation_1-auc:0.76568
[86]	validation_0-auc:0.86788	validation_1-auc:0.76572
[87]	validation_0-auc:0.86858	validation_1-auc:0.76584
[88]	valid

[0]	validation_0-auc:0.74150	validation_1-auc:0.72221
[1]	validation_0-auc:0.75017	validation_1-auc:0.72798
[2]	validation_0-auc:0.75586	validation_1-auc:0.73124
[3]	validation_0-auc:0.75990	validation_1-auc:0.73199
[4]	validation_0-auc:0.76358	validation_1-auc:0.73363
[5]	validation_0-auc:0.76695	validation_1-auc:0.73446
[6]	validation_0-auc:0.76905	validation_1-auc:0.73478
[7]	validation_0-auc:0.77115	validation_1-auc:0.73492
[8]	validation_0-auc:0.77234	validation_1-auc:0.73484
[9]	validation_0-auc:0.77408	validation_1-auc:0.73628
[10]	validation_0-auc:0.77552	validation_1-auc:0.73665
[11]	validation_0-auc:0.77686	validation_1-auc:0.73696
[12]	validation_0-auc:0.77950	validation_1-auc:0.73784
[13]	validation_0-auc:0.78113	validation_1-auc:0.73838
[14]	validation_0-auc:0.78298	validation_1-auc:0.73863
[15]	validation_0-auc:0.78466	validation_1-auc:0.73946
[16]	validation_0-auc:0.78621	validation_1-auc:0.73929
[17]	validation_0-auc:0.78769	validation_1-auc:0.73998
[18]	validation_0-au

### pos_cashから特徴量を作成

In [37]:
pos = pos_cash_balance.copy()
pos = baseline.reduce_mem_usage(pos)
print(pos.shape)
pos.head()

Memory usage of dataframe is 610.43 MB
Memory usage of optimization is 290039510.00 MB
Decreased by -47513511.5%
(10001358, 8)


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [38]:
pos_ohe = pd.get_dummies(pos, columns=['NAME_CONTRACT_STATUS'], dummy_na=True)
col_ohe = sorted(list(set(pos_ohe.columns) - set(pos.columns)))
print(len(col_ohe))
pos_ohe.head()

10


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,SK_DPD,SK_DPD_DEF,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Amortized debt,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Canceled,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Returned to the store,NAME_CONTRACT_STATUS_Signed,NAME_CONTRACT_STATUS_XNA,NAME_CONTRACT_STATUS_nan
0,1803195,182943,-31,48.0,45.0,0,0,1,0,0,0,0,0,0,0,0,0
1,1715348,367990,-33,36.0,35.0,0,0,1,0,0,0,0,0,0,0,0,0
2,1784872,397406,-32,12.0,9.0,0,0,1,0,0,0,0,0,0,0,0,0
3,1903291,269225,-35,48.0,42.0,0,0,1,0,0,0,0,0,0,0,0,0
4,2341044,334279,-35,36.0,35.0,0,0,1,0,0,0,0,0,0,0,0,0


In [39]:
# Some new features 2022/06/14
pos_ohe['pos CNT_INSTALMENT_more_CNT_INSTALMENT_FUTURE'] = (pos_ohe['CNT_INSTALMENT'] > pos_ohe['CNT_INSTALMENT_FUTURE']).astype(int)
pos_ohe.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,SK_DPD,SK_DPD_DEF,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Amortized debt,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Canceled,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Returned to the store,NAME_CONTRACT_STATUS_Signed,NAME_CONTRACT_STATUS_XNA,NAME_CONTRACT_STATUS_nan,pos CNT_INSTALMENT_more_CNT_INSTALMENT_FUTURE
0,1803195,182943,-31,48.0,45.0,0,0,1,0,0,0,0,0,0,0,0,0,1
1,1715348,367990,-33,36.0,35.0,0,0,1,0,0,0,0,0,0,0,0,0,1
2,1784872,397406,-32,12.0,9.0,0,0,1,0,0,0,0,0,0,0,0,0,1
3,1903291,269225,-35,48.0,42.0,0,0,1,0,0,0,0,0,0,0,0,0,1
4,2341044,334279,-35,36.0,35.0,0,0,1,0,0,0,0,0,0,0,0,0,1


In [40]:
pos_ohe_agg = pos_ohe.groupby('SK_ID_CURR').agg({
    "MONTHS_BALANCE": ['min', 'max', 'size', 'mean', 'var', 'sum']
    ,"CNT_INSTALMENT": ['min', 'max', 'size', 'mean', 'var', 'sum']
    ,"CNT_INSTALMENT_FUTURE": ['min', 'max', 'size', 'mean', 'var', 'sum']
    ,"SK_DPD": ['min', 'max', 'size', 'mean', 'var', 'sum']
    ,"SK_DPD_DEF": ['min', 'max', 'size', 'mean', 'var', 'sum']
    # カテゴリ変数をone-hot-encodingした値の集約
    ,"NAME_CONTRACT_STATUS_Active": ["mean"]
    ,"NAME_CONTRACT_STATUS_Amortized debt": ["mean"]
    ,"NAME_CONTRACT_STATUS_Approved": ["mean"]
    ,"NAME_CONTRACT_STATUS_Canceled": ["mean"]
    ,"NAME_CONTRACT_STATUS_Completed": ["mean"]
    ,"NAME_CONTRACT_STATUS_Demand": ["mean"]
    ,"NAME_CONTRACT_STATUS_Returned to the store": ["mean"]
    ,"NAME_CONTRACT_STATUS_Signed": ["mean"]
    ,"NAME_CONTRACT_STATUS_XNA": ["mean"]
    ,"NAME_CONTRACT_STATUS_nan": ["mean"]
    # IDのユニーク数をカウント
    ,"SK_ID_PREV":["count", "nunique"]
})

# カラム名の付与
pos_ohe_agg.columns = [i + "_" + j for i,j in pos_ohe_agg.columns]
pos_ohe_agg = pos_ohe_agg.reset_index(drop=False)

print(pos_ohe_agg.shape)
pos_ohe_agg.head()

(337252, 43)


,SK_ID_CURR,MONTHS_BALANCE_min,MONTHS_BALANCE_max,MONTHS_BALANCE_size,MONTHS_BALANCE_mean,MONTHS_BALANCE_var,MONTHS_BALANCE_sum,CNT_INSTALMENT_min,CNT_INSTALMENT_max,CNT_INSTALMENT_size,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100001,-96,-53,9,-72.555556,435.277778,-653.0,4.0,4.0,9,...,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,0.0,9,2
1,100002,-19,-1,19,-10.000000,31.666667,-190.0,24.0,24.0,19,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,19,1
2,100003,-77,-18,28,-43.785714,607.137566,-1226.0,6.0,12.0,28,...,0.0,0.0,0.071429,0.0,0.0,0.000000,0.0,0.0,28,3
3,100004,-27,-24,4,-25.500000,1.666667,-102.0,3.0,4.0,4,...,0.0,0.0,0.250000,0.0,0.0,0.000000,0.0,0.0,4,1
4,100005,-25,-15,11,-20.000000,11.000000,-220.0,9.0,12.0,11,...,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0.0,11,1


In [41]:
df_train = pd.merge(app_train, pos_ohe_agg, on='SK_ID_CURR', how='left')
print(df_train.shape)
df_train.head()

(307511, 336)


,SK_ID_CURR,TARGET,CODE_GENDER,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_INCOME_TYPE,NAME_FAMILY_STATUS,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,1,1,0,202500.0,406597.5,24700.5,351000.0,0,1,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,19.0,1.0
1,100003,0,0,0,270000.0,1293502.5,35698.5,1129500.0,3,0,...,0.0,0.0,0.071429,0.0,0.000000,0.000000,0.0,0.0,28.0,3.0
2,100004,0,1,0,67500.0,135000.0,6750.0,135000.0,0,1,...,0.0,0.0,0.250000,0.0,0.000000,0.000000,0.0,0.0,4.0,1.0
3,100006,0,0,0,135000.0,312682.5,29686.5,297000.0,0,2,...,0.0,0.0,0.095238,0.0,0.047619,0.000000,0.0,0.0,21.0,3.0
4,100007,0,1,0,121500.0,513000.0,21865.5,513000.0,0,1,...,0.0,0.0,0.045455,0.0,0.000000,0.015152,0.0,0.0,66.0,5.0


In [42]:
# infをnanへ置換
df_train = df_train.replace([np.inf, -np.inf], np.nan)
#app_test = app_test.replace([np.inf, -np.inf], np.nan)

In [43]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

In [44]:
baseline = Baseline('SK_ID_CURR')

train_oof, imp, metrics = baseline.train_xgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 334) (61503, 334)
[21:41:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_state", "silent", "subsumple" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-auc:0.73769	validation_1-auc:0.72279
[1]	validation_0-auc:0.75039	validation_1-auc:0.73380
[2]	validation_0-auc:0.75654	validation_1-auc:0.73782
[3]	validation_0-auc:0.76078	validation_1-auc:0.73942
[4]	validation_0-auc:0.76424	validation_1-auc:0.73999
[5]	validation_0-auc:0.76633	validation_1-auc:0.74090
[6]	validation_0-auc:0.76958	validation_1-auc:0.74196
[7]	validation_0-auc:0.77236	validation_1-auc:0.74263
[8]	validation_0-auc:0.77495	validation_1-auc:0.74

[30]	validation_0-auc:0.81284	validation_1-auc:0.75529
[31]	validation_0-auc:0.81440	validation_1-auc:0.75585
[32]	validation_0-auc:0.81598	validation_1-auc:0.75675
[33]	validation_0-auc:0.81763	validation_1-auc:0.75738
[34]	validation_0-auc:0.81898	validation_1-auc:0.75799
[35]	validation_0-auc:0.82075	validation_1-auc:0.75857
[36]	validation_0-auc:0.82234	validation_1-auc:0.75883
[37]	validation_0-auc:0.82432	validation_1-auc:0.75976
[38]	validation_0-auc:0.82608	validation_1-auc:0.76051
[39]	validation_0-auc:0.82770	validation_1-auc:0.76101
[40]	validation_0-auc:0.82926	validation_1-auc:0.76106
[41]	validation_0-auc:0.83110	validation_1-auc:0.76155
[42]	validation_0-auc:0.83279	validation_1-auc:0.76200
[43]	validation_0-auc:0.83425	validation_1-auc:0.76235
[44]	validation_0-auc:0.83533	validation_1-auc:0.76292
[45]	validation_0-auc:0.83665	validation_1-auc:0.76348
[46]	validation_0-auc:0.83823	validation_1-auc:0.76370
[47]	validation_0-auc:0.83968	validation_1-auc:0.76390
[48]	valid

[70]	validation_0-auc:0.86555	validation_1-auc:0.77049
[71]	validation_0-auc:0.86638	validation_1-auc:0.77065
[72]	validation_0-auc:0.86708	validation_1-auc:0.77088
[73]	validation_0-auc:0.86833	validation_1-auc:0.77096
[74]	validation_0-auc:0.86928	validation_1-auc:0.77108
[75]	validation_0-auc:0.86988	validation_1-auc:0.77099
[76]	validation_0-auc:0.87090	validation_1-auc:0.77085
[77]	validation_0-auc:0.87172	validation_1-auc:0.77091
[78]	validation_0-auc:0.87231	validation_1-auc:0.77094
[79]	validation_0-auc:0.87305	validation_1-auc:0.77091
[80]	validation_0-auc:0.87375	validation_1-auc:0.77104
[81]	validation_0-auc:0.87482	validation_1-auc:0.77102
[82]	validation_0-auc:0.87541	validation_1-auc:0.77107
[83]	validation_0-auc:0.87629	validation_1-auc:0.77119
[84]	validation_0-auc:0.87680	validation_1-auc:0.77108
[85]	validation_0-auc:0.87764	validation_1-auc:0.77115
[86]	validation_0-auc:0.87839	validation_1-auc:0.77119
[87]	validation_0-auc:0.87889	validation_1-auc:0.77109
[88]	valid

[0]	validation_0-auc:0.73809	validation_1-auc:0.72493
[1]	validation_0-auc:0.75170	validation_1-auc:0.73235
[2]	validation_0-auc:0.75788	validation_1-auc:0.73466
[3]	validation_0-auc:0.76268	validation_1-auc:0.73584
[4]	validation_0-auc:0.76480	validation_1-auc:0.73744
[5]	validation_0-auc:0.76702	validation_1-auc:0.73781
[6]	validation_0-auc:0.77004	validation_1-auc:0.73908
[7]	validation_0-auc:0.77281	validation_1-auc:0.73979
[8]	validation_0-auc:0.77585	validation_1-auc:0.74050
[9]	validation_0-auc:0.77809	validation_1-auc:0.74180
[10]	validation_0-auc:0.77921	validation_1-auc:0.74244
[11]	validation_0-auc:0.78077	validation_1-auc:0.74322
[12]	validation_0-auc:0.78257	validation_1-auc:0.74363
[13]	validation_0-auc:0.78402	validation_1-auc:0.74380
[14]	validation_0-auc:0.78605	validation_1-auc:0.74504
[15]	validation_0-auc:0.78719	validation_1-auc:0.74502
[16]	validation_0-auc:0.78907	validation_1-auc:0.74527
[17]	validation_0-auc:0.79130	validation_1-auc:0.74553
[18]	validation_0-au

In [45]:
imp.sort_values("imp", ascending=False)[:10]

,col,imp,imp_std
60,EXT_SOURCE_mean,0.030612,0.001517
176,NEW_EXT_SOURCES_GEO,0.029510,0.001117
59,EXT_SOURCE_max,0.018075,0.001732
178,NEW_EXT_SOURCES_MEDIAN,0.017525,0.001412
179,NEW_EXT_SOURCES_PROD,0.009620,0.001527
329,app EXT_SOURCE_2_times_EXT_SOURCE_3,0.008388,0.000805
144,NAME_EDUCATION_TYPE_Higher education,0.007827,0.000465
173,NEW_DOC_IND_KURT,0.007492,0.000406
34,CODE_GENDER,0.007133,0.000253
88,FLAG_OWN_CAR_Y,0.006631,0.001667


In [46]:
# テーブル結合
df_test = pd.merge(app_test, pos_ohe_agg, on='SK_ID_CURR', how="left")

# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [47]:
test_pred = baseline.predict_xgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [48]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_XgbFeatureEngineering2.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.032955
1,100005,0.088096
2,100013,0.023195
3,100028,0.030593
4,100038,0.166922


### credit card blanceの読み込み

In [49]:
credit = credit_card_balance_origin.copy()
cerdit = baseline.reduce_mem_usage(credit)
print(credit.shape)
credit.head()

Memory usage of dataframe is 673.88 MB
Memory usage of optimization is 334107272.00 MB
Decreased by -49579318.7%
(3840312, 23)


,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970001,135000,0.0,877.5,0.0,877.5,1700.324951,...,0.000000,0.000000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.554688,45000,2250.0,2250.0,0.0,0.0,2250.000000,...,64875.554688,64875.554688,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.224609,450000,0.0,0.0,0.0,0.0,2250.000000,...,31460.085938,31460.085938,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.109375,225000,2250.0,2250.0,0.0,0.0,11795.759766,...,233048.968750,233048.968750,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.468750,450000,0.0,11547.0,0.0,11547.0,22924.890625,...,453919.468750,453919.468750,0.0,1,0.0,1.0,101.0,Active,0,0


In [50]:
# Some new features
cerdit['card_missing'] = cerdit.isnull().sum(axis = 1).values
cerdit['card_SK_DPD-MONTHS_BALANCE'] = cerdit['SK_DPD'] - cerdit['MONTHS_BALANCE']
cerdit['card_SK_DPD_DEF-MONTHS_BALANCE'] = cerdit['SK_DPD_DEF'] - cerdit['MONTHS_BALANCE']
cerdit['card_SK_DPD-SK_DPD_DEF'] = cerdit['SK_DPD'] - cerdit['SK_DPD_DEF']

cerdit['card_AMT_TOTAL_RECEIVABLE-AMT_RECIVABLE'] = cerdit['AMT_TOTAL_RECEIVABLE'] - cerdit['AMT_RECIVABLE']
cerdit['card_AMT_TOTAL_RECEIVABLE-AMT_RECEIVABLE_PRINCIPAL'] = cerdit['AMT_TOTAL_RECEIVABLE'] - cerdit['AMT_RECEIVABLE_PRINCIPAL']
cerdit['card_AMT_RECIVABLE-AMT_RECEIVABLE_PRINCIPAL'] = cerdit['AMT_RECIVABLE'] - cerdit['AMT_RECEIVABLE_PRINCIPAL']

cerdit['card_AMT_BALANCE-AMT_RECIVABLE'] = cerdit['AMT_BALANCE'] - cerdit['AMT_RECIVABLE']
cerdit['card_AMT_BALANCE-AMT_RECEIVABLE_PRINCIPAL'] = cerdit['AMT_BALANCE'] - cerdit['AMT_RECEIVABLE_PRINCIPAL']
cerdit['card_AMT_BALANCE-AMT_TOTAL_RECEIVABLE'] = cerdit['AMT_BALANCE'] - cerdit['AMT_TOTAL_RECEIVABLE']

cerdit['card_AMT_DRAWINGS_CURRENT-AMT_DRAWINGS_ATM_CURRENT'] = cerdit['AMT_DRAWINGS_CURRENT'] - cerdit['AMT_DRAWINGS_ATM_CURRENT']
cerdit['card_AMT_DRAWINGS_CURRENT-AMT_DRAWINGS_OTHER_CURRENT'] = cerdit['AMT_DRAWINGS_CURRENT'] - cerdit['AMT_DRAWINGS_OTHER_CURRENT']
cerdit['card_AMT_DRAWINGS_CURRENT-AMT_DRAWINGS_POS_CURRENT'] = cerdit['AMT_DRAWINGS_CURRENT'] - cerdit['AMT_DRAWINGS_POS_CURRENT']

In [51]:
# one-hotencoding(処理対象カラムを取得)
encoding_columns = []

for col in credit.columns:
    if credit[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

#encoding_columns

NAME_CONTRACT_STATUS


In [52]:
# one-hot-encoding
for col in encoding_columns:
    credit_ohe = pd.get_dummies(credit, columns=[col], dummy_na=True)
    col_ohe = sorted(list(set(credit_ohe.columns) - set(credit.columns)))

print(len(col_ohe))
col_ohe

8


['NAME_CONTRACT_STATUS_Active',
 'NAME_CONTRACT_STATUS_Approved',
 'NAME_CONTRACT_STATUS_Completed',
 'NAME_CONTRACT_STATUS_Demand',
 'NAME_CONTRACT_STATUS_Refused',
 'NAME_CONTRACT_STATUS_Sent proposal',
 'NAME_CONTRACT_STATUS_Signed',
 'NAME_CONTRACT_STATUS_nan']

In [53]:
print(credit.columns)

Index(['SK_ID_PREV', 'SK_ID_CURR', 'MONTHS_BALANCE', 'AMT_BALANCE',
       'AMT_CREDIT_LIMIT_ACTUAL', 'AMT_DRAWINGS_ATM_CURRENT',
       'AMT_DRAWINGS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT',
       'AMT_DRAWINGS_POS_CURRENT', 'AMT_INST_MIN_REGULARITY',
       'AMT_PAYMENT_CURRENT', 'AMT_PAYMENT_TOTAL_CURRENT',
       'AMT_RECEIVABLE_PRINCIPAL', 'AMT_RECIVABLE', 'AMT_TOTAL_RECEIVABLE',
       'CNT_DRAWINGS_ATM_CURRENT', 'CNT_DRAWINGS_CURRENT',
       'CNT_DRAWINGS_OTHER_CURRENT', 'CNT_DRAWINGS_POS_CURRENT',
       'CNT_INSTALMENT_MATURE_CUM', 'NAME_CONTRACT_STATUS', 'SK_DPD',
       'SK_DPD_DEF', 'card_missing', 'card_SK_DPD-MONTHS_BALANCE',
       'card_SK_DPD_DEF-MONTHS_BALANCE', 'card_SK_DPD-SK_DPD_DEF',
       'card_AMT_TOTAL_RECEIVABLE-AMT_RECIVABLE',
       'card_AMT_TOTAL_RECEIVABLE-AMT_RECEIVABLE_PRINCIPAL',
       'card_AMT_RECIVABLE-AMT_RECEIVABLE_PRINCIPAL',
       'card_AMT_BALANCE-AMT_RECIVABLE',
       'card_AMT_BALANCE-AMT_RECEIVABLE_PRINCIPAL',
       'card_AMT_BALANCE-A

In [54]:
credit_ohe_agg = credit_ohe.groupby('SK_ID_CURR').agg(
    {
        # 数値変数の集約
        'SK_ID_CURR': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'MONTHS_BALANCE': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'AMT_BALANCE': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'AMT_CREDIT_LIMIT_ACTUAL': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'AMT_DRAWINGS_ATM_CURRENT': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'AMT_DRAWINGS_CURRENT': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'AMT_DRAWINGS_OTHER_CURRENT': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'AMT_DRAWINGS_POS_CURRENT': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'AMT_INST_MIN_REGULARITY': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'AMT_PAYMENT_CURRENT': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'AMT_PAYMENT_TOTAL_CURRENT': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'AMT_RECEIVABLE_PRINCIPAL': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'AMT_RECIVABLE': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'AMT_TOTAL_RECEIVABLE': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'CNT_DRAWINGS_ATM_CURRENT': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'CNT_DRAWINGS_CURRENT': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'CNT_DRAWINGS_OTHER_CURRENT': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'CNT_DRAWINGS_POS_CURRENT': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'CNT_INSTALMENT_MATURE_CUM': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'SK_DPD': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'SK_DPD_DEF': ['min', 'max', 'size', 'mean', 'var', 'sum']
        # 
        ,'NAME_CONTRACT_STATUS_Active': ['mean']
        ,'NAME_CONTRACT_STATUS_Approved': ['mean']
        ,'NAME_CONTRACT_STATUS_Completed': ['mean']
        ,'NAME_CONTRACT_STATUS_Demand': ['mean']
        ,'NAME_CONTRACT_STATUS_Refused': ['mean']
        ,'NAME_CONTRACT_STATUS_Sent proposal': ['mean']
        ,'NAME_CONTRACT_STATUS_Signed': ['mean']
        ,'NAME_CONTRACT_STATUS_nan': ['mean']
        # IDのユニーク数をカウント
        ,"SK_ID_PREV":["count", "nunique"]
    }
)

# カラム名の付与
credit_ohe_agg.columns = [i + "_" + j for i,j in credit_ohe_agg.columns]
credit_ohe_agg = credit_ohe_agg.reset_index(drop=False)

print(credit_ohe_agg.shape)
credit_ohe_agg.head()

(103558, 137)


,SK_ID_CURR,SK_ID_CURR_min,SK_ID_CURR_max,SK_ID_CURR_size,SK_ID_CURR_mean,SK_ID_CURR_var,SK_ID_CURR_sum,MONTHS_BALANCE_min,MONTHS_BALANCE_max,MONTHS_BALANCE_size,...,NAME_CONTRACT_STATUS_Active_mean,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Refused_mean,NAME_CONTRACT_STATUS_Sent proposal_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100006,100006,100006,6,100006.0,0.0,600036,-6,-1,6,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,6,1
1,100011,100011,100011,74,100011.0,0.0,7400814,-75,-2,74,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,74,1
2,100013,100013,100013,96,100013.0,0.0,9601248,-96,-1,96,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,96,1
3,100021,100021,100021,17,100021.0,0.0,1700357,-18,-2,17,...,0.411765,0.0,0.588235,0.0,0.0,0.0,0.0,0.0,17,1
4,100023,100023,100023,8,100023.0,0.0,800184,-11,-4,8,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,8,1


In [55]:
# infをnanへ置換
credit_ohe_agg = credit_ohe_agg.replace([np.inf, -np.inf], np.nan)

In [56]:
df_train = pd.merge(app_train, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, pos_ohe_agg, on='SK_ID_CURR', how='left')
print(df_train.shape)
df_train.head()

(307511, 472)


,SK_ID_CURR,TARGET,CODE_GENDER,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_INCOME_TYPE,NAME_FAMILY_STATUS,...,NAME_CONTRACT_STATUS_Approved_mean_y,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean_y,NAME_CONTRACT_STATUS_Demand_mean_y,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean_y,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean_y,SK_ID_PREV_count_y,SK_ID_PREV_nunique_y
0,100002,1,1,0,202500.0,406597.5,24700.5,351000.0,0,1,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,19.0,1.0
1,100003,0,0,0,270000.0,1293502.5,35698.5,1129500.0,3,0,...,0.0,0.0,0.071429,0.0,0.000000,0.000000,0.0,0.0,28.0,3.0
2,100004,0,1,0,67500.0,135000.0,6750.0,135000.0,0,1,...,0.0,0.0,0.250000,0.0,0.000000,0.000000,0.0,0.0,4.0,1.0
3,100006,0,0,0,135000.0,312682.5,29686.5,297000.0,0,2,...,0.0,0.0,0.095238,0.0,0.047619,0.000000,0.0,0.0,21.0,3.0
4,100007,0,1,0,121500.0,513000.0,21865.5,513000.0,0,1,...,0.0,0.0,0.045455,0.0,0.000000,0.015152,0.0,0.0,66.0,5.0


In [57]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

In [58]:
baseline = Baseline('SK_ID_CURR')

train_oof, imp, metrics = baseline.train_xgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 470) (61503, 470)
[21:42:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_state", "silent", "subsumple" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-auc:0.74016	validation_1-auc:0.72265
[1]	validation_0-auc:0.75356	validation_1-auc:0.73082
[2]	validation_0-auc:0.76068	validation_1-auc:0.73483
[3]	validation_0-auc:0.76531	validation_1-auc:0.73725
[4]	validation_0-auc:0.76892	validation_1-auc:0.73979
[5]	validation_0-auc:0.77157	validation_1-auc:0.74194
[6]	validation_0-auc:0.77393	validation_1-auc:0.74271
[7]	validation_0-auc:0.77598	validation_1-auc:0.74397
[8]	validation_0-auc:0.77803	validation_1-auc:0.74

[30]	validation_0-auc:0.81658	validation_1-auc:0.75869
[31]	validation_0-auc:0.81836	validation_1-auc:0.75955
[32]	validation_0-auc:0.82012	validation_1-auc:0.76042
[33]	validation_0-auc:0.82186	validation_1-auc:0.76107
[34]	validation_0-auc:0.82346	validation_1-auc:0.76153
[35]	validation_0-auc:0.82484	validation_1-auc:0.76206
[36]	validation_0-auc:0.82686	validation_1-auc:0.76232
[37]	validation_0-auc:0.82856	validation_1-auc:0.76317
[38]	validation_0-auc:0.83010	validation_1-auc:0.76353
[39]	validation_0-auc:0.83189	validation_1-auc:0.76394
[40]	validation_0-auc:0.83352	validation_1-auc:0.76458
[41]	validation_0-auc:0.83504	validation_1-auc:0.76493
[42]	validation_0-auc:0.83653	validation_1-auc:0.76551
[43]	validation_0-auc:0.83836	validation_1-auc:0.76582
[44]	validation_0-auc:0.83963	validation_1-auc:0.76641
[45]	validation_0-auc:0.84126	validation_1-auc:0.76636
[46]	validation_0-auc:0.84242	validation_1-auc:0.76693
[47]	validation_0-auc:0.84378	validation_1-auc:0.76716
[48]	valid

[70]	validation_0-auc:0.86805	validation_1-auc:0.77277
[71]	validation_0-auc:0.86894	validation_1-auc:0.77292
[72]	validation_0-auc:0.86979	validation_1-auc:0.77297
[73]	validation_0-auc:0.87040	validation_1-auc:0.77334
[74]	validation_0-auc:0.87108	validation_1-auc:0.77329
[75]	validation_0-auc:0.87187	validation_1-auc:0.77340
[76]	validation_0-auc:0.87292	validation_1-auc:0.77342
[77]	validation_0-auc:0.87377	validation_1-auc:0.77347
[78]	validation_0-auc:0.87455	validation_1-auc:0.77366
[79]	validation_0-auc:0.87544	validation_1-auc:0.77377
[80]	validation_0-auc:0.87624	validation_1-auc:0.77376
[81]	validation_0-auc:0.87713	validation_1-auc:0.77397
[82]	validation_0-auc:0.87838	validation_1-auc:0.77397
[83]	validation_0-auc:0.87900	validation_1-auc:0.77404
[84]	validation_0-auc:0.87979	validation_1-auc:0.77397
[85]	validation_0-auc:0.88026	validation_1-auc:0.77401
[86]	validation_0-auc:0.88086	validation_1-auc:0.77405
[87]	validation_0-auc:0.88144	validation_1-auc:0.77417
[88]	valid

[0]	validation_0-auc:0.74028	validation_1-auc:0.72083
[1]	validation_0-auc:0.75367	validation_1-auc:0.72973
[2]	validation_0-auc:0.76128	validation_1-auc:0.73224
[3]	validation_0-auc:0.76587	validation_1-auc:0.73471
[4]	validation_0-auc:0.76912	validation_1-auc:0.73675
[5]	validation_0-auc:0.77219	validation_1-auc:0.73898
[6]	validation_0-auc:0.77461	validation_1-auc:0.74025
[7]	validation_0-auc:0.77730	validation_1-auc:0.74154
[8]	validation_0-auc:0.77898	validation_1-auc:0.74192
[9]	validation_0-auc:0.78174	validation_1-auc:0.74322
[10]	validation_0-auc:0.78308	validation_1-auc:0.74350
[11]	validation_0-auc:0.78431	validation_1-auc:0.74372
[12]	validation_0-auc:0.78601	validation_1-auc:0.74460
[13]	validation_0-auc:0.78807	validation_1-auc:0.74500
[14]	validation_0-auc:0.78950	validation_1-auc:0.74537
[15]	validation_0-auc:0.79082	validation_1-auc:0.74568
[16]	validation_0-auc:0.79221	validation_1-auc:0.74662
[17]	validation_0-auc:0.79395	validation_1-auc:0.74686
[18]	validation_0-au

In [59]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
162,EXT_SOURCE_mean,0.028819,0.003136
292,NEW_EXT_SOURCES_GEO,0.025525,0.002363
294,NEW_EXT_SOURCES_MEDIAN,0.013357,0.000888
161,EXT_SOURCE_max,0.009303,0.001015
260,NAME_EDUCATION_TYPE_Higher education,0.006537,0.000974
465,app EXT_SOURCE_2_times_EXT_SOURCE_3,0.005899,0.000581
163,EXT_SOURCE_min,0.005765,0.000096
295,NEW_EXT_SOURCES_PROD,0.005654,0.001407
289,NEW_DOC_IND_KURT,0.005527,0.000513
136,CODE_GENDER,0.005031,0.000339


In [60]:
# テーブル結合
df_test = pd.merge(app_test, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, pos_ohe_agg, on='SK_ID_CURR', how='left')

# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [61]:
test_pred = baseline.predict_xgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [62]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_XgbFeatureEngineering3.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.051692
1,100005,0.091839
2,100013,0.026519
3,100028,0.034237
4,100038,0.196949


### previous applicationの読み込み

In [63]:
previous = previous_application_origin.copy()
previous = baseline.reduce_mem_usage(previous)
print(previous.shape)
previous.head()

Memory usage of dataframe is 471.48 MB
Memory usage of optimization is 340723784.00 MB
Decreased by -72266632.8%
(1670214, 37)


,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430054,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615234,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735352,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335938,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.394531,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
# Some new features
previous['prev_missing'] = previous.isnull().sum(axis = 1).values
previous['prev_AMT_APPLICATION_div_AMT_CREDIT'] = previous['AMT_APPLICATION'] / previous['AMT_CREDIT']
previous['prev_AMT_APPLICATION-AMT_CREDIT'] = previous['AMT_APPLICATION'] - previous['AMT_CREDIT']
previous['prev_AMT_APPLICATION-AMT_GOODS_PRICE'] = previous['AMT_APPLICATION'] - previous['AMT_GOODS_PRICE']
previous['prev_AMT_GOODS_PRICE-AMT_CREDIT'] = previous['AMT_GOODS_PRICE'] - previous['AMT_CREDIT']
previous['prev_DAYS_FIRST_DRAWING-DAYS_FIRST_DUE'] = previous['DAYS_FIRST_DRAWING'] - previous['DAYS_FIRST_DUE']
previous['prev_DAYS_TERMINATION_less-500'] = (previous['DAYS_TERMINATION'] < -500).astype(int)

In [65]:
# one-hotencoding(処理対象カラムを取得)
encoding_columns = []

for col in previous.columns:
    if previous[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

#encoding_columns

NAME_CONTRACT_TYPE
WEEKDAY_APPR_PROCESS_START
FLAG_LAST_APPL_PER_CONTRACT
NAME_CASH_LOAN_PURPOSE
NAME_CONTRACT_STATUS
NAME_PAYMENT_TYPE
CODE_REJECT_REASON
NAME_TYPE_SUITE
NAME_CLIENT_TYPE
NAME_GOODS_CATEGORY
NAME_PORTFOLIO
NAME_PRODUCT_TYPE
CHANNEL_TYPE
NAME_SELLER_INDUSTRY
NAME_YIELD_GROUP
PRODUCT_COMBINATION


In [66]:
# one-hot-encoding
for col in encoding_columns:
    previous_ohe = pd.get_dummies(previous, columns=[col], dummy_na=True)
    col_ohe = sorted(list(set(previous_ohe.columns) - set(previous.columns)))

print(len(col_ohe))
col_ohe

18


['PRODUCT_COMBINATION_Card Street',
 'PRODUCT_COMBINATION_Card X-Sell',
 'PRODUCT_COMBINATION_Cash',
 'PRODUCT_COMBINATION_Cash Street: high',
 'PRODUCT_COMBINATION_Cash Street: low',
 'PRODUCT_COMBINATION_Cash Street: middle',
 'PRODUCT_COMBINATION_Cash X-Sell: high',
 'PRODUCT_COMBINATION_Cash X-Sell: low',
 'PRODUCT_COMBINATION_Cash X-Sell: middle',
 'PRODUCT_COMBINATION_POS household with interest',
 'PRODUCT_COMBINATION_POS household without interest',
 'PRODUCT_COMBINATION_POS industry with interest',
 'PRODUCT_COMBINATION_POS industry without interest',
 'PRODUCT_COMBINATION_POS mobile with interest',
 'PRODUCT_COMBINATION_POS mobile without interest',
 'PRODUCT_COMBINATION_POS other with interest',
 'PRODUCT_COMBINATION_POS others without interest',
 'PRODUCT_COMBINATION_nan']

In [67]:
print(previous.columns)

Index(['SK_ID_PREV', 'SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'AMT_ANNUITY',
       'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE',
       'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START',
       'FLAG_LAST_APPL_PER_CONTRACT', 'NFLAG_LAST_APPL_IN_DAY',
       'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY',
       'RATE_INTEREST_PRIVILEGED', 'NAME_CASH_LOAN_PURPOSE',
       'NAME_CONTRACT_STATUS', 'DAYS_DECISION', 'NAME_PAYMENT_TYPE',
       'CODE_REJECT_REASON', 'NAME_TYPE_SUITE', 'NAME_CLIENT_TYPE',
       'NAME_GOODS_CATEGORY', 'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE',
       'CHANNEL_TYPE', 'SELLERPLACE_AREA', 'NAME_SELLER_INDUSTRY',
       'CNT_PAYMENT', 'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION',
       'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION',
       'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL',
       'prev_missing', 'prev_AMT_APPLICATION_div_AMT_CREDIT',
       'prev_AMT_APPLICATION-AMT_CREDIT',
       'prev_AMT

In [68]:
previous_ohe_agg = credit_ohe.groupby('SK_ID_CURR').agg(
    {
        # 数値変数の集約
        'MONTHS_BALANCE': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'AMT_BALANCE': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'AMT_CREDIT_LIMIT_ACTUAL': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'AMT_DRAWINGS_ATM_CURRENT': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'AMT_DRAWINGS_CURRENT': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'AMT_DRAWINGS_OTHER_CURRENT': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'AMT_DRAWINGS_POS_CURRENT': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'AMT_INST_MIN_REGULARITY': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'AMT_PAYMENT_CURRENT': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'AMT_PAYMENT_TOTAL_CURRENT': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'AMT_RECEIVABLE_PRINCIPAL': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'AMT_RECIVABLE': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'AMT_TOTAL_RECEIVABLE': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'CNT_DRAWINGS_ATM_CURRENT': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'CNT_DRAWINGS_CURRENT': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'CNT_DRAWINGS_OTHER_CURRENT': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'CNT_DRAWINGS_POS_CURRENT': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'CNT_INSTALMENT_MATURE_CUM': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'SK_DPD': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'SK_DPD_DEF': ['min', 'max', 'size', 'mean', 'var', 'sum']
        # 
        ,'NAME_CONTRACT_STATUS_Active': ['mean']
        ,'NAME_CONTRACT_STATUS_Approved': ['mean']
        ,'NAME_CONTRACT_STATUS_Completed': ['mean']
        ,'NAME_CONTRACT_STATUS_Demand': ['mean']
        ,'NAME_CONTRACT_STATUS_Refused': ['mean']
        ,'NAME_CONTRACT_STATUS_Sent proposal': ['mean']
        ,'NAME_CONTRACT_STATUS_Signed': ['mean']
        ,'NAME_CONTRACT_STATUS_nan': ['mean']
        # IDのユニーク数をカウント
        ,"SK_ID_PREV":["count", "nunique"]
    }
)

# カラム名の付与
previous_ohe_agg.columns = [i + "_" + j for i,j in previous_ohe_agg.columns]
previous_ohe_agg = previous_ohe_agg.reset_index(drop=False)

print(previous_ohe_agg.shape)
previous_ohe_agg.head()

(103558, 131)


,SK_ID_CURR,MONTHS_BALANCE_min,MONTHS_BALANCE_max,MONTHS_BALANCE_size,MONTHS_BALANCE_mean,MONTHS_BALANCE_var,MONTHS_BALANCE_sum,AMT_BALANCE_min,AMT_BALANCE_max,AMT_BALANCE_size,...,NAME_CONTRACT_STATUS_Active_mean,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Refused_mean,NAME_CONTRACT_STATUS_Sent proposal_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100006,-6,-1,6,-3.5,3.5,-21.0,0.0,0.00000,6,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,6,1
1,100011,-75,-2,74,-38.5,462.5,-2849.0,0.0,189000.00000,74,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,74,1
2,100013,-96,-1,96,-48.5,776.0,-4656.0,0.0,161420.21875,96,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,96,1
3,100021,-18,-2,17,-10.0,25.5,-170.0,0.0,0.00000,17,...,0.411765,0.0,0.588235,0.0,0.0,0.0,0.0,0.0,17,1
4,100023,-11,-4,8,-7.5,6.0,-60.0,0.0,0.00000,8,...,1.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,8,1


In [69]:
# infをnanへ置換
previous_ohe_agg = previous_ohe_agg.replace([np.inf, -np.inf], np.nan)

In [70]:
df_train = pd.merge(app_train, previous_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, pos_ohe_agg, on='SK_ID_CURR', how='left')
print(df_train.shape)
df_train.head()

(307511, 602)


,SK_ID_CURR,TARGET,CODE_GENDER,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_INCOME_TYPE,NAME_FAMILY_STATUS,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,1,1,0,202500.0,406597.5,24700.5,351000.0,0,1,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,19.0,1.0
1,100003,0,0,0,270000.0,1293502.5,35698.5,1129500.0,3,0,...,0.0,0.0,0.071429,0.0,0.000000,0.000000,0.0,0.0,28.0,3.0
2,100004,0,1,0,67500.0,135000.0,6750.0,135000.0,0,1,...,0.0,0.0,0.250000,0.0,0.000000,0.000000,0.0,0.0,4.0,1.0
3,100006,0,0,0,135000.0,312682.5,29686.5,297000.0,0,2,...,0.0,0.0,0.095238,0.0,0.047619,0.000000,0.0,0.0,21.0,3.0
4,100007,0,1,0,121500.0,513000.0,21865.5,513000.0,0,1,...,0.0,0.0,0.045455,0.0,0.000000,0.015152,0.0,0.0,66.0,5.0


In [71]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

for col in x_train.columns:
    if x_train[col].dtype=="O":
        x_train[col] = x_train[col].astype('category')

In [72]:
baseline = Baseline('SK_ID_CURR')

train_oof, imp, metrics = baseline.train_xgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 600) (61503, 600)
[21:44:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_state", "silent", "subsumple" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-auc:0.74057	validation_1-auc:0.71971
[1]	validation_0-auc:0.75295	validation_1-auc:0.73052
[2]	validation_0-auc:0.75817	validation_1-auc:0.73301
[3]	validation_0-auc:0.76239	validation_1-auc:0.73524
[4]	validation_0-auc:0.76634	validation_1-auc:0.73636
[5]	validation_0-auc:0.76937	validation_1-auc:0.73765
[6]	validation_0-auc:0.77196	validation_1-auc:0.73944
[7]	validation_0-auc:0.77460	validation_1-auc:0.74087
[8]	validation_0-auc:0.77692	validation_1-auc:0.74

[30]	validation_0-auc:0.81624	validation_1-auc:0.75957
[31]	validation_0-auc:0.81786	validation_1-auc:0.75998
[32]	validation_0-auc:0.81964	validation_1-auc:0.76049
[33]	validation_0-auc:0.82139	validation_1-auc:0.76114
[34]	validation_0-auc:0.82332	validation_1-auc:0.76164
[35]	validation_0-auc:0.82486	validation_1-auc:0.76181
[36]	validation_0-auc:0.82649	validation_1-auc:0.76239
[37]	validation_0-auc:0.82818	validation_1-auc:0.76282
[38]	validation_0-auc:0.83001	validation_1-auc:0.76327
[39]	validation_0-auc:0.83161	validation_1-auc:0.76411
[40]	validation_0-auc:0.83267	validation_1-auc:0.76442
[41]	validation_0-auc:0.83437	validation_1-auc:0.76473
[42]	validation_0-auc:0.83613	validation_1-auc:0.76522
[43]	validation_0-auc:0.83790	validation_1-auc:0.76558
[44]	validation_0-auc:0.83961	validation_1-auc:0.76619
[45]	validation_0-auc:0.84111	validation_1-auc:0.76648
[46]	validation_0-auc:0.84285	validation_1-auc:0.76705
[47]	validation_0-auc:0.84412	validation_1-auc:0.76731
[48]	valid

[70]	validation_0-auc:0.86906	validation_1-auc:0.77295
[71]	validation_0-auc:0.86970	validation_1-auc:0.77300
[72]	validation_0-auc:0.87075	validation_1-auc:0.77324
[73]	validation_0-auc:0.87182	validation_1-auc:0.77312
[74]	validation_0-auc:0.87262	validation_1-auc:0.77325
[75]	validation_0-auc:0.87346	validation_1-auc:0.77327
[76]	validation_0-auc:0.87421	validation_1-auc:0.77337
[77]	validation_0-auc:0.87506	validation_1-auc:0.77350
[78]	validation_0-auc:0.87547	validation_1-auc:0.77351
[79]	validation_0-auc:0.87644	validation_1-auc:0.77354
[80]	validation_0-auc:0.87713	validation_1-auc:0.77367
[81]	validation_0-auc:0.87750	validation_1-auc:0.77382
[82]	validation_0-auc:0.87862	validation_1-auc:0.77386
[83]	validation_0-auc:0.87938	validation_1-auc:0.77392
[84]	validation_0-auc:0.88001	validation_1-auc:0.77381
[85]	validation_0-auc:0.88099	validation_1-auc:0.77375
[86]	validation_0-auc:0.88167	validation_1-auc:0.77398
[87]	validation_0-auc:0.88195	validation_1-auc:0.77404
[88]	valid

[0]	validation_0-auc:0.74055	validation_1-auc:0.71812
[1]	validation_0-auc:0.75490	validation_1-auc:0.72821
[2]	validation_0-auc:0.76012	validation_1-auc:0.73177
[3]	validation_0-auc:0.76443	validation_1-auc:0.73291
[4]	validation_0-auc:0.76813	validation_1-auc:0.73506
[5]	validation_0-auc:0.77088	validation_1-auc:0.73615
[6]	validation_0-auc:0.77338	validation_1-auc:0.73806
[7]	validation_0-auc:0.77602	validation_1-auc:0.73959
[8]	validation_0-auc:0.77795	validation_1-auc:0.74067
[9]	validation_0-auc:0.78002	validation_1-auc:0.74128
[10]	validation_0-auc:0.78215	validation_1-auc:0.74268
[11]	validation_0-auc:0.78346	validation_1-auc:0.74352
[12]	validation_0-auc:0.78463	validation_1-auc:0.74409
[13]	validation_0-auc:0.78631	validation_1-auc:0.74418
[14]	validation_0-auc:0.78857	validation_1-auc:0.74435
[15]	validation_0-auc:0.79044	validation_1-auc:0.74501
[16]	validation_0-auc:0.79235	validation_1-auc:0.74571
[17]	validation_0-auc:0.79367	validation_1-auc:0.74628
[18]	validation_0-au

In [73]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
408,NEW_EXT_SOURCES_GEO,0.023513,0.001656
264,EXT_SOURCE_mean,0.017687,0.001698
410,NEW_EXT_SOURCES_MEDIAN,0.013762,0.001065
411,NEW_EXT_SOURCES_PROD,0.008369,0.000717
265,EXT_SOURCE_min,0.005264,0.000188
595,app EXT_SOURCE_2_times_EXT_SOURCE_3,0.005071,0.000780
376,NAME_EDUCATION_TYPE_Higher education,0.005032,0.000409
405,NEW_DOC_IND_KURT,0.004764,0.000218
238,CODE_GENDER,0.004386,0.000155
379,NAME_EDUCATION_TYPE_Secondary / secondary special,0.004260,0.000382


In [74]:
# テーブル結合
df_test = pd.merge(app_test, previous_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, pos_ohe_agg, on='SK_ID_CURR', how='left')

# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

# カテゴリ変数をcategory型へ変換
for col in x_test.columns:
    if x_test[col].dtype=="O":
        x_test[col] = x_test[col].astype('category')

In [75]:
test_pred = baseline.predict_xgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [76]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_XgbFeatureEngineering4.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.036012
1,100005,0.095238
2,100013,0.024580
3,100028,0.034184
4,100038,0.200949


### installments paymentsの読み込み

In [77]:
installments = installments_payments_origin.copy()
installments = baseline.reduce_mem_usage(installments)
print(installments.shape)
installments.head()

Memory usage of dataframe is 830.41 MB
Memory usage of optimization is 408162158.00 MB
Decreased by -49151908.2%
(13605401, 8)


,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.359863,6948.359863
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525024,1716.525024
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000000,25425.000000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130859,24350.130859
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040039,2160.584961


In [78]:
# Some new features
installments['ins_DAYS_ENTRY_PAYMENT-DAYS_INSTALMENT'] = installments['DAYS_ENTRY_PAYMENT'] - installments['DAYS_INSTALMENT']
installments['ins_NUM_INSTALMENT_NUMBER_100'] = (installments['NUM_INSTALMENT_NUMBER'] == 100).astype(int)
installments['ins_DAYS_INSTALMENT_more_NUM_INSTALMENT_NUMBER'] = (installments['DAYS_INSTALMENT'] > installments['NUM_INSTALMENT_NUMBER'] * 50 / 3 - 11500 / 3).astype(int)
installments['ins_AMT_INSTALMENT-AMT_PAYMENT'] = installments['AMT_INSTALMENT'] - installments['AMT_PAYMENT']
installments['ins_AMT_PAYMENT_div_AMT_INSTALMENT'] = installments['AMT_PAYMENT'] / installments['AMT_INSTALMENT']

In [79]:
# one-hotencoding(処理対象カラムを取得)
encoding_columns = []

for col in installments.columns:
    if installments[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

encoding_columns

[]

In [80]:
# one-hot-encoding
#for col in encoding_columns:
#    installments_ohe = pd.get_dummies(installments, columns=[col], dummy_na=True)
#    col_ohe = sorted(list(set(installments_ohe.columns) - set(installments.columns)))

#print(len(col_ohe))
#col_ohe
installments_ohe = installments.copy()

In [81]:
print(installments.columns)

Index(['SK_ID_PREV', 'SK_ID_CURR', 'NUM_INSTALMENT_VERSION',
       'NUM_INSTALMENT_NUMBER', 'DAYS_INSTALMENT', 'DAYS_ENTRY_PAYMENT',
       'AMT_INSTALMENT', 'AMT_PAYMENT',
       'ins_DAYS_ENTRY_PAYMENT-DAYS_INSTALMENT',
       'ins_NUM_INSTALMENT_NUMBER_100',
       'ins_DAYS_INSTALMENT_more_NUM_INSTALMENT_NUMBER',
       'ins_AMT_INSTALMENT-AMT_PAYMENT', 'ins_AMT_PAYMENT_div_AMT_INSTALMENT'],
      dtype='object')


In [82]:
installments_ohe = installments_ohe.rename(columns={'SK_ID_PREV': 'SK_ID_PREV_IP'})

In [83]:
installments_ohe_agg = installments_ohe.groupby('SK_ID_CURR').agg(
    {
        # 数値変数の集約
        'NUM_INSTALMENT_VERSION': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'NUM_INSTALMENT_NUMBER': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'DAYS_INSTALMENT': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'DAYS_ENTRY_PAYMENT': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'AMT_INSTALMENT': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'AMT_PAYMENT': ['min', 'max', 'size', 'mean', 'var', 'sum']
        # IDのユニーク数をカウント
        ,"SK_ID_PREV_IP":["count", "nunique"]
    }
)

# カラム名の付与
installments_ohe_agg.columns = [i + "_" + j for i,j in installments_ohe_agg.columns]
installments_ohe_agg = installments_ohe_agg.reset_index(drop=False)

print(installments_ohe_agg.shape)
installments_ohe_agg.head()

(339587, 39)


,SK_ID_CURR,NUM_INSTALMENT_VERSION_min,NUM_INSTALMENT_VERSION_max,NUM_INSTALMENT_VERSION_size,NUM_INSTALMENT_VERSION_mean,NUM_INSTALMENT_VERSION_var,NUM_INSTALMENT_VERSION_sum,NUM_INSTALMENT_NUMBER_min,NUM_INSTALMENT_NUMBER_max,NUM_INSTALMENT_NUMBER_size,...,AMT_INSTALMENT_var,AMT_INSTALMENT_sum,AMT_PAYMENT_min,AMT_PAYMENT_max,AMT_PAYMENT_size,AMT_PAYMENT_mean,AMT_PAYMENT_var,AMT_PAYMENT_sum,SK_ID_PREV_IP_count,SK_ID_PREV_IP_nunique
0,100001,1.0,2.0,7,1.142857,0.142857,8.0,1,4,7,...,2.577265e+07,4.119593e+04,3951.000000,17397.900391,7,5885.132324,2.577265e+07,4.119593e+04,7,2
1,100002,1.0,2.0,19,1.052632,0.052632,20.0,1,19,19,...,1.011641e+08,2.196257e+05,9251.775391,53093.746094,19,11559.247070,1.011641e+08,2.196257e+05,19,1
2,100003,1.0,2.0,25,1.040000,0.040000,26.0,1,12,25,...,1.221967e+10,1.618865e+06,6662.970215,560835.375000,25,64754.585938,1.221967e+10,1.618865e+06,25,3
3,100004,1.0,2.0,3,1.333333,0.333333,4.0,1,3,3,...,9.071371e+06,2.128846e+04,5357.250000,10573.964844,3,7096.154785,9.071371e+06,2.128846e+04,3,1
4,100005,1.0,2.0,9,1.111111,0.111111,10.0,1,9,9,...,1.832709e+07,5.616184e+04,4813.200195,17656.244141,9,6240.205078,1.832709e+07,5.616184e+04,9,1


In [84]:
# infをnanへ置換
installments_ohe_agg = installments_ohe_agg.replace([np.inf, -np.inf], np.nan)

In [85]:
df_train = pd.merge(app_train, installments_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, previous_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, pos_ohe_agg, on='SK_ID_CURR', how='left')
print(df_train.shape)
df_train.head()

(307511, 640)


,SK_ID_CURR,TARGET,CODE_GENDER,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_INCOME_TYPE,NAME_FAMILY_STATUS,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,1,1,0,202500.0,406597.5,24700.5,351000.0,0,1,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,19.0,1.0
1,100003,0,0,0,270000.0,1293502.5,35698.5,1129500.0,3,0,...,0.0,0.0,0.071429,0.0,0.000000,0.000000,0.0,0.0,28.0,3.0
2,100004,0,1,0,67500.0,135000.0,6750.0,135000.0,0,1,...,0.0,0.0,0.250000,0.0,0.000000,0.000000,0.0,0.0,4.0,1.0
3,100006,0,0,0,135000.0,312682.5,29686.5,297000.0,0,2,...,0.0,0.0,0.095238,0.0,0.047619,0.000000,0.0,0.0,21.0,3.0
4,100007,0,1,0,121500.0,513000.0,21865.5,513000.0,0,1,...,0.0,0.0,0.045455,0.0,0.000000,0.015152,0.0,0.0,66.0,5.0


In [86]:
#df_train.to_csv(
#    path_or_buf="./home-credit-default-risk/submit/df_train.csv" # 出力先
#    ,index=None                                        # indexの出力有無
#)
len(list(df_train.columns))

640

In [87]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

In [88]:
baseline = Baseline('SK_ID_CURR')

train_oof, imp, metrics = baseline.train_xgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 638) (61503, 638)
[21:46:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_state", "silent", "subsumple" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-auc:0.74078	validation_1-auc:0.72126
[1]	validation_0-auc:0.75443	validation_1-auc:0.72929
[2]	validation_0-auc:0.76049	validation_1-auc:0.73207
[3]	validation_0-auc:0.76493	validation_1-auc:0.73575
[4]	validation_0-auc:0.76852	validation_1-auc:0.73792
[5]	validation_0-auc:0.77203	validation_1-auc:0.73975
[6]	validation_0-auc:0.77511	validation_1-auc:0.74098
[7]	validation_0-auc:0.77714	validation_1-auc:0.74212
[8]	validation_0-auc:0.77939	validation_1-auc:0.74

[30]	validation_0-auc:0.82024	validation_1-auc:0.75974
[31]	validation_0-auc:0.82205	validation_1-auc:0.76049
[32]	validation_0-auc:0.82355	validation_1-auc:0.76088
[33]	validation_0-auc:0.82508	validation_1-auc:0.76154
[34]	validation_0-auc:0.82721	validation_1-auc:0.76216
[35]	validation_0-auc:0.82909	validation_1-auc:0.76314
[36]	validation_0-auc:0.83087	validation_1-auc:0.76353
[37]	validation_0-auc:0.83263	validation_1-auc:0.76413
[38]	validation_0-auc:0.83416	validation_1-auc:0.76484
[39]	validation_0-auc:0.83571	validation_1-auc:0.76543
[40]	validation_0-auc:0.83721	validation_1-auc:0.76582
[41]	validation_0-auc:0.83925	validation_1-auc:0.76624
[42]	validation_0-auc:0.84042	validation_1-auc:0.76685
[43]	validation_0-auc:0.84217	validation_1-auc:0.76748
[44]	validation_0-auc:0.84392	validation_1-auc:0.76841
[45]	validation_0-auc:0.84545	validation_1-auc:0.76880
[46]	validation_0-auc:0.84711	validation_1-auc:0.76939
[47]	validation_0-auc:0.84850	validation_1-auc:0.76966
[48]	valid

[70]	validation_0-auc:0.87433	validation_1-auc:0.77536
[71]	validation_0-auc:0.87539	validation_1-auc:0.77564
[72]	validation_0-auc:0.87646	validation_1-auc:0.77568
[73]	validation_0-auc:0.87707	validation_1-auc:0.77565
[74]	validation_0-auc:0.87774	validation_1-auc:0.77550
[75]	validation_0-auc:0.87858	validation_1-auc:0.77545
[76]	validation_0-auc:0.87891	validation_1-auc:0.77560
[77]	validation_0-auc:0.88008	validation_1-auc:0.77583
[78]	validation_0-auc:0.88086	validation_1-auc:0.77604
[79]	validation_0-auc:0.88186	validation_1-auc:0.77613
[80]	validation_0-auc:0.88292	validation_1-auc:0.77610
[81]	validation_0-auc:0.88347	validation_1-auc:0.77629
[82]	validation_0-auc:0.88427	validation_1-auc:0.77634
[83]	validation_0-auc:0.88528	validation_1-auc:0.77644
[84]	validation_0-auc:0.88611	validation_1-auc:0.77650
[85]	validation_0-auc:0.88707	validation_1-auc:0.77664
[86]	validation_0-auc:0.88753	validation_1-auc:0.77670
[87]	validation_0-auc:0.88841	validation_1-auc:0.77681
[88]	valid

[0]	validation_0-auc:0.73991	validation_1-auc:0.72168
[1]	validation_0-auc:0.75422	validation_1-auc:0.72998
[2]	validation_0-auc:0.76109	validation_1-auc:0.73370
[3]	validation_0-auc:0.76550	validation_1-auc:0.73658
[4]	validation_0-auc:0.76838	validation_1-auc:0.73774
[5]	validation_0-auc:0.77180	validation_1-auc:0.73902
[6]	validation_0-auc:0.77489	validation_1-auc:0.74036
[7]	validation_0-auc:0.77758	validation_1-auc:0.74231
[8]	validation_0-auc:0.77994	validation_1-auc:0.74345
[9]	validation_0-auc:0.78172	validation_1-auc:0.74429
[10]	validation_0-auc:0.78334	validation_1-auc:0.74459
[11]	validation_0-auc:0.78494	validation_1-auc:0.74483
[12]	validation_0-auc:0.78650	validation_1-auc:0.74561
[13]	validation_0-auc:0.78862	validation_1-auc:0.74601
[14]	validation_0-auc:0.79064	validation_1-auc:0.74710
[15]	validation_0-auc:0.79222	validation_1-auc:0.74791
[16]	validation_0-auc:0.79415	validation_1-auc:0.74825
[17]	validation_0-auc:0.79573	validation_1-auc:0.74882
[18]	validation_0-au

In [89]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
432,NEW_EXT_SOURCES_GEO,0.018057,0.001041
288,EXT_SOURCE_mean,0.014212,0.001017
289,EXT_SOURCE_min,0.013204,0.002107
287,EXT_SOURCE_max,0.009978,0.000713
434,NEW_EXT_SOURCES_MEDIAN,0.009482,0.000603
435,NEW_EXT_SOURCES_PROD,0.006667,0.000584
633,app EXT_SOURCE_2_times_EXT_SOURCE_3,0.005531,0.000637
400,NAME_EDUCATION_TYPE_Higher education,0.005281,0.000205
429,NEW_DOC_IND_KURT,0.004148,0.000430
250,CODE_GENDER,0.004041,0.000306


In [90]:
df_test = pd.merge(app_test, installments_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, previous_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, pos_ohe_agg, on='SK_ID_CURR', how='left')

# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

len(list(df_test.columns))

639

In [91]:
test_pred = baseline.predict_xgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [92]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_XgbFeatureEngineering5.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.044559
1,100005,0.100369
2,100013,0.032156
3,100028,0.034977
4,100038,0.146493


### bureauの読み込み

In [93]:
bureau = bureau_origin.copy()
bureau = baseline.reduce_mem_usage(bureau)
print(bureau.shape)
bureau.head()

Memory usage of dataframe is 222.62 MB
Memory usage of optimization is 125299372.00 MB
Decreased by -56283785.5%
(1716428, 17)


,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [94]:
### 
for col in bureau.columns:
    bureau[col] = bureau[col].replace('\s', '_', regex=True)
    bureau[col] = bureau[col].replace(':', '_', regex=True)
    bureau[col] = bureau[col].replace('/', '_', regex=True)
    
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency_1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer_credit,-131,NaN
1,215354,5714463,Active,currency_1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit_card,-20,NaN
2,215354,5714464,Active,currency_1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer_credit,-16,NaN
3,215354,5714465,Active,currency_1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit_card,-16,NaN
4,215354,5714466,Active,currency_1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer_credit,-21,NaN


In [95]:
# one-hotencoding(処理対象カラムを取得)
encoding_columns = []

for col in bureau.columns:
    if bureau[col].dtype=="O":
        print(col)
        encoding_columns.append(col)

CREDIT_ACTIVE
CREDIT_CURRENCY
CREDIT_TYPE


In [96]:
# one-hot-encoding
bureau_ohe = pd.get_dummies(bureau, columns=['CREDIT_ACTIVE'], dummy_na=True)
col_ohe = sorted(list(set(bureau_ohe.columns) - set(bureau.columns)))

bureau_ohe = pd.get_dummies(bureau_ohe, columns=['CREDIT_CURRENCY'], dummy_na=True)
col_ohe = sorted(list(set(bureau_ohe.columns) - set(bureau.columns)))

bureau_ohe = pd.get_dummies(bureau_ohe, columns=['CREDIT_TYPE'], dummy_na=True)
col_ohe = sorted(list(set(bureau_ohe.columns) - set(bureau.columns)))

print(len(col_ohe))
col_ohe

26


['CREDIT_ACTIVE_Active',
 'CREDIT_ACTIVE_Bad_debt',
 'CREDIT_ACTIVE_Closed',
 'CREDIT_ACTIVE_Sold',
 'CREDIT_ACTIVE_nan',
 'CREDIT_CURRENCY_currency_1',
 'CREDIT_CURRENCY_currency_2',
 'CREDIT_CURRENCY_currency_3',
 'CREDIT_CURRENCY_currency_4',
 'CREDIT_CURRENCY_nan',
 'CREDIT_TYPE_Another_type_of_loan',
 'CREDIT_TYPE_Car_loan',
 'CREDIT_TYPE_Cash_loan_(non-earmarked)',
 'CREDIT_TYPE_Consumer_credit',
 'CREDIT_TYPE_Credit_card',
 'CREDIT_TYPE_Interbank_credit',
 'CREDIT_TYPE_Loan_for_business_development',
 'CREDIT_TYPE_Loan_for_purchase_of_shares_(margin_lending)',
 'CREDIT_TYPE_Loan_for_the_purchase_of_equipment',
 'CREDIT_TYPE_Loan_for_working_capital_replenishment',
 'CREDIT_TYPE_Microloan',
 'CREDIT_TYPE_Mobile_operator_loan',
 'CREDIT_TYPE_Mortgage',
 'CREDIT_TYPE_Real_estate_loan',
 'CREDIT_TYPE_Unknown_type_of_loan',
 'CREDIT_TYPE_nan']

In [97]:
print(bureau.columns)

Index(['SK_ID_CURR', 'SK_ID_BUREAU', 'CREDIT_ACTIVE', 'CREDIT_CURRENCY',
       'DAYS_CREDIT', 'CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_ENDDATE',
       'DAYS_ENDDATE_FACT', 'AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG',
       'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT',
       'AMT_CREDIT_SUM_OVERDUE', 'CREDIT_TYPE', 'DAYS_CREDIT_UPDATE',
       'AMT_ANNUITY'],
      dtype='object')


In [98]:
print(bureau_ohe.columns)

Index(['SK_ID_CURR', 'SK_ID_BUREAU', 'DAYS_CREDIT', 'CREDIT_DAY_OVERDUE',
       'DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT', 'AMT_CREDIT_MAX_OVERDUE',
       'CNT_CREDIT_PROLONG', 'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT',
       'AMT_CREDIT_SUM_LIMIT', 'AMT_CREDIT_SUM_OVERDUE', 'DAYS_CREDIT_UPDATE',
       'AMT_ANNUITY', 'CREDIT_ACTIVE_Active', 'CREDIT_ACTIVE_Bad_debt',
       'CREDIT_ACTIVE_Closed', 'CREDIT_ACTIVE_Sold', 'CREDIT_ACTIVE_nan',
       'CREDIT_CURRENCY_currency_1', 'CREDIT_CURRENCY_currency_2',
       'CREDIT_CURRENCY_currency_3', 'CREDIT_CURRENCY_currency_4',
       'CREDIT_CURRENCY_nan', 'CREDIT_TYPE_Another_type_of_loan',
       'CREDIT_TYPE_Car_loan', 'CREDIT_TYPE_Cash_loan_(non-earmarked)',
       'CREDIT_TYPE_Consumer_credit', 'CREDIT_TYPE_Credit_card',
       'CREDIT_TYPE_Interbank_credit',
       'CREDIT_TYPE_Loan_for_business_development',
       'CREDIT_TYPE_Loan_for_purchase_of_shares_(margin_lending)',
       'CREDIT_TYPE_Loan_for_the_purchase_of_equipment',
  

In [99]:
bureau_ohe = bureau_ohe.rename(columns={'SK_ID_BUREAU': 'SK_ID_BUREAU_BUREAU'})

In [100]:
bureau_ohe.head()

,SK_ID_CURR,SK_ID_BUREAU_BUREAU,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,...,CREDIT_TYPE_Loan_for_business_development,CREDIT_TYPE_Loan_for_purchase_of_shares_(margin_lending),CREDIT_TYPE_Loan_for_the_purchase_of_equipment,CREDIT_TYPE_Loan_for_working_capital_replenishment,CREDIT_TYPE_Microloan,CREDIT_TYPE_Mobile_operator_loan,CREDIT_TYPE_Mortgage,CREDIT_TYPE_Real_estate_loan,CREDIT_TYPE_Unknown_type_of_loan,CREDIT_TYPE_nan
0,215354,5714462,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,215354,5714463,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,...,0,0,0,0,0,0,0,0,0,0
2,215354,5714464,-203,0,528.0,NaN,NaN,0,464323.5,NaN,...,0,0,0,0,0,0,0,0,0,0
3,215354,5714465,-203,0,NaN,NaN,NaN,0,90000.0,NaN,...,0,0,0,0,0,0,0,0,0,0
4,215354,5714466,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,...,0,0,0,0,0,0,0,0,0,0


In [101]:
bureau_ohe_agg = bureau_ohe.groupby('SK_ID_CURR').agg(
    {
        # 数値変数の集約
        'DAYS_CREDIT': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'CREDIT_DAY_OVERDUE': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'DAYS_CREDIT_ENDDATE': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'DAYS_ENDDATE_FACT': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'AMT_CREDIT_MAX_OVERDUE': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'CNT_CREDIT_PROLONG': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'AMT_CREDIT_SUM': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'AMT_CREDIT_SUM_DEBT': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'AMT_CREDIT_SUM_LIMIT': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'AMT_CREDIT_SUM_OVERDUE': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'DAYS_CREDIT_UPDATE': ['min', 'max', 'size', 'mean', 'var', 'sum']
        ,'AMT_ANNUITY': ['min', 'max', 'size', 'mean', 'var', 'sum']
        # 
        ,'CREDIT_TYPE_Another_type_of_loan': ['mean']
        ,'CREDIT_TYPE_Car_loan': ['mean']
        ,'CREDIT_TYPE_Cash_loan_(non-earmarked)': ['mean']
        ,'CREDIT_TYPE_Consumer_credit': ['mean']
        ,'CREDIT_TYPE_Credit_card': ['mean']
        ,'CREDIT_TYPE_Interbank_credit': ['mean']
        ,'CREDIT_TYPE_Loan_for_business_development': ['mean']
        ,'CREDIT_TYPE_Loan_for_purchase_of_shares_(margin_lending)': ['mean']
        ,'CREDIT_TYPE_Loan_for_the_purchase_of_equipment': ['mean']
        ,'CREDIT_TYPE_Loan_for_working_capital_replenishment': ['mean']
        ,'CREDIT_TYPE_Microloan': ['mean']
        ,'CREDIT_TYPE_Mobile_operator_loan': ['mean']
        ,'CREDIT_TYPE_Mortgage': ['mean']
        ,'CREDIT_TYPE_Real_estate_loan': ['mean']
        ,'CREDIT_TYPE_Unknown_type_of_loan': ['mean']
        ,'CREDIT_TYPE_nan': ['mean']
        ,'CREDIT_ACTIVE_Active': ['mean']
        ,'CREDIT_ACTIVE_Bad_debt': ['mean']
        ,'CREDIT_ACTIVE_Closed': ['mean']
        ,'CREDIT_ACTIVE_Sold': ['mean']
        ,'CREDIT_ACTIVE_nan': ['mean']
        ,'CREDIT_CURRENCY_currency_1': ['mean']
        ,'CREDIT_CURRENCY_currency_2': ['mean']
        ,'CREDIT_CURRENCY_currency_3': ['mean']
        ,'CREDIT_CURRENCY_currency_4': ['mean']
        ,'CREDIT_CURRENCY_nan': ['mean']
        ,'CREDIT_TYPE_Another_type_of_loan': ['mean']
        ,'CREDIT_TYPE_Car_loan': ['mean']
        ,'CREDIT_TYPE_Cash_loan_(non-earmarked)': ['mean']
        ,'CREDIT_TYPE_Consumer_credit': ['mean']
        ,'CREDIT_TYPE_Credit_card': ['mean']
        ,'CREDIT_TYPE_Interbank_credit': ['mean']
        ,'CREDIT_TYPE_Loan_for_business_development': ['mean']
        ,'CREDIT_TYPE_Loan_for_purchase_of_shares_(margin_lending)': ['mean']
        ,'CREDIT_TYPE_Loan_for_the_purchase_of_equipment': ['mean']
        ,'CREDIT_TYPE_Loan_for_working_capital_replenishment': ['mean']
        ,'CREDIT_TYPE_Microloan': ['mean']
        ,'CREDIT_TYPE_Mobile_operator_loan': ['mean']
        ,'CREDIT_TYPE_Mortgage': ['mean']
        ,'CREDIT_TYPE_Real_estate_loan': ['mean']
        ,'CREDIT_TYPE_Unknown_type_of_loan': ['mean']
        ,'CREDIT_TYPE_nan': ['mean']
        # IDのユニーク数をカウント
        ,'SK_ID_BUREAU_BUREAU': ["count", "nunique"]
    }
)

# カラム名の付与
bureau_ohe_agg.columns = [i + "_" + j for i,j in bureau_ohe_agg.columns]
bureau_ohe_agg = bureau_ohe_agg.reset_index(drop=False)

#
bureau_ohe_agg['debt_credit_ratio_None'] = bureau_ohe_agg['AMT_CREDIT_SUM_DEBT_sum'] / bureau_ohe_agg['AMT_CREDIT_SUM_sum']


print(bureau_ohe_agg.shape)
bureau_ohe_agg.head()

(305811, 102)


,SK_ID_CURR,DAYS_CREDIT_min,DAYS_CREDIT_max,DAYS_CREDIT_size,DAYS_CREDIT_mean,DAYS_CREDIT_var,DAYS_CREDIT_sum,CREDIT_DAY_OVERDUE_min,CREDIT_DAY_OVERDUE_max,CREDIT_DAY_OVERDUE_size,...,CREDIT_ACTIVE_Sold_mean,CREDIT_ACTIVE_nan_mean,CREDIT_CURRENCY_currency_1_mean,CREDIT_CURRENCY_currency_2_mean,CREDIT_CURRENCY_currency_3_mean,CREDIT_CURRENCY_currency_4_mean,CREDIT_CURRENCY_nan_mean,SK_ID_BUREAU_BUREAU_count,SK_ID_BUREAU_BUREAU_nunique,debt_credit_ratio_None
0,100001,-1572,-49,7,-735.000000,240043.666667,-5145.0,0,0,7,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,7,7,0.410555
1,100002,-1437,-103,8,-874.000000,186150.000000,-6992.0,0,0,8,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,8,8,0.284122
2,100003,-2586,-606,4,-1400.750000,827783.583333,-5603.0,0,0,4,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4,4,0.000000
3,100004,-1326,-408,2,-867.000000,421362.000000,-1734.0,0,0,2,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2,2,0.000000
4,100005,-373,-62,3,-190.666667,26340.333333,-572.0,0,0,3,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3,3,0.864992


In [102]:
# infをnanへ置換
bureau_ohe_agg = bureau_ohe_agg.replace([np.inf, -np.inf], np.nan)

In [103]:
df_train = pd.merge(app_train, bureau_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, installments_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, previous_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, pos_ohe_agg, on='SK_ID_CURR', how='left')
print(df_train.shape)
df_train.head()

(307511, 741)


,SK_ID_CURR,TARGET,CODE_GENDER,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_INCOME_TYPE,NAME_FAMILY_STATUS,...,NAME_CONTRACT_STATUS_Approved_mean,NAME_CONTRACT_STATUS_Canceled_mean,NAME_CONTRACT_STATUS_Completed_mean,NAME_CONTRACT_STATUS_Demand_mean,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique
0,100002,1,1,0,202500.0,406597.5,24700.5,351000.0,0,1,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,19.0,1.0
1,100003,0,0,0,270000.0,1293502.5,35698.5,1129500.0,3,0,...,0.0,0.0,0.071429,0.0,0.000000,0.000000,0.0,0.0,28.0,3.0
2,100004,0,1,0,67500.0,135000.0,6750.0,135000.0,0,1,...,0.0,0.0,0.250000,0.0,0.000000,0.000000,0.0,0.0,4.0,1.0
3,100006,0,0,0,135000.0,312682.5,29686.5,297000.0,0,2,...,0.0,0.0,0.095238,0.0,0.047619,0.000000,0.0,0.0,21.0,3.0
4,100007,0,1,0,121500.0,513000.0,21865.5,513000.0,0,1,...,0.0,0.0,0.045455,0.0,0.000000,0.015152,0.0,0.0,66.0,5.0


In [104]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

In [105]:
baseline = Baseline('SK_ID_CURR')

train_oof, imp, metrics = baseline.train_xgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 739) (61503, 739)
[21:48:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_state", "silent", "subsumple" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-auc:0.74040	validation_1-auc:0.72031
[1]	validation_0-auc:0.75401	validation_1-auc:0.72994
[2]	validation_0-auc:0.76131	validation_1-auc:0.73494
[3]	validation_0-auc:0.76627	validation_1-auc:0.73764
[4]	validation_0-auc:0.76857	validation_1-auc:0.73989
[5]	validation_0-auc:0.77144	validation_1-auc:0.74076
[6]	validation_0-auc:0.77412	validation_1-auc:0.74303
[7]	validation_0-auc:0.77824	validation_1-auc:0.74482
[8]	validation_0-auc:0.77991	validation_1-auc:0.74

[30]	validation_0-auc:0.82565	validation_1-auc:0.76278
[31]	validation_0-auc:0.82760	validation_1-auc:0.76344
[32]	validation_0-auc:0.82874	validation_1-auc:0.76382
[33]	validation_0-auc:0.83051	validation_1-auc:0.76426
[34]	validation_0-auc:0.83267	validation_1-auc:0.76499
[35]	validation_0-auc:0.83458	validation_1-auc:0.76577
[36]	validation_0-auc:0.83654	validation_1-auc:0.76613
[37]	validation_0-auc:0.83806	validation_1-auc:0.76723
[38]	validation_0-auc:0.84045	validation_1-auc:0.76778
[39]	validation_0-auc:0.84209	validation_1-auc:0.76842
[40]	validation_0-auc:0.84388	validation_1-auc:0.76886
[41]	validation_0-auc:0.84589	validation_1-auc:0.76929
[42]	validation_0-auc:0.84747	validation_1-auc:0.76960
[43]	validation_0-auc:0.84920	validation_1-auc:0.77033
[44]	validation_0-auc:0.85087	validation_1-auc:0.77072
[45]	validation_0-auc:0.85252	validation_1-auc:0.77102
[46]	validation_0-auc:0.85397	validation_1-auc:0.77173
[47]	validation_0-auc:0.85539	validation_1-auc:0.77204
[48]	valid

[70]	validation_0-auc:0.88069	validation_1-auc:0.78001
[71]	validation_0-auc:0.88166	validation_1-auc:0.78006
[72]	validation_0-auc:0.88278	validation_1-auc:0.78025
[73]	validation_0-auc:0.88349	validation_1-auc:0.78040
[74]	validation_0-auc:0.88405	validation_1-auc:0.78070
[75]	validation_0-auc:0.88496	validation_1-auc:0.78077
[76]	validation_0-auc:0.88587	validation_1-auc:0.78099
[77]	validation_0-auc:0.88654	validation_1-auc:0.78090
[78]	validation_0-auc:0.88741	validation_1-auc:0.78111
[79]	validation_0-auc:0.88833	validation_1-auc:0.78124
[80]	validation_0-auc:0.88919	validation_1-auc:0.78131
[81]	validation_0-auc:0.88983	validation_1-auc:0.78142
[82]	validation_0-auc:0.89087	validation_1-auc:0.78147
[83]	validation_0-auc:0.89154	validation_1-auc:0.78162
[84]	validation_0-auc:0.89269	validation_1-auc:0.78158
[85]	validation_0-auc:0.89345	validation_1-auc:0.78162
[86]	validation_0-auc:0.89422	validation_1-auc:0.78178
[87]	validation_0-auc:0.89483	validation_1-auc:0.78189
[88]	valid

[0]	validation_0-auc:0.74029	validation_1-auc:0.72008
[1]	validation_0-auc:0.75288	validation_1-auc:0.72831
[2]	validation_0-auc:0.76307	validation_1-auc:0.73484
[3]	validation_0-auc:0.76641	validation_1-auc:0.73696
[4]	validation_0-auc:0.76890	validation_1-auc:0.73844
[5]	validation_0-auc:0.77230	validation_1-auc:0.74076
[6]	validation_0-auc:0.77507	validation_1-auc:0.74180
[7]	validation_0-auc:0.77787	validation_1-auc:0.74216
[8]	validation_0-auc:0.78045	validation_1-auc:0.74298
[9]	validation_0-auc:0.78337	validation_1-auc:0.74441
[10]	validation_0-auc:0.78582	validation_1-auc:0.74532
[11]	validation_0-auc:0.78741	validation_1-auc:0.74603
[12]	validation_0-auc:0.78991	validation_1-auc:0.74688
[13]	validation_0-auc:0.79254	validation_1-auc:0.74784
[14]	validation_0-auc:0.79464	validation_1-auc:0.74832
[15]	validation_0-auc:0.79658	validation_1-auc:0.74894
[16]	validation_0-auc:0.79858	validation_1-auc:0.74909
[17]	validation_0-auc:0.80073	validation_1-auc:0.74945
[18]	validation_0-au

In [106]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
530,NEW_EXT_SOURCES_GEO,0.025658,0.003057
386,EXT_SOURCE_mean,0.014065,0.001145
532,NEW_EXT_SOURCES_MEDIAN,0.009844,0.000748
385,EXT_SOURCE_max,0.007030,0.000691
387,EXT_SOURCE_min,0.006546,0.000536
498,NAME_EDUCATION_TYPE_Higher education,0.005175,0.000342
733,app EXT_SOURCE_2_times_EXT_SOURCE_3,0.004078,0.000805
292,CODE_GENDER,0.003849,0.000288
527,NEW_DOC_IND_KURT,0.003676,0.000467
501,NAME_EDUCATION_TYPE_Secondary / secondary special,0.003489,0.000314


In [107]:
df_test = pd.merge(app_test, bureau_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, installments_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, previous_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, pos_ohe_agg, on='SK_ID_CURR', how='left')

# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

len(list(df_test.columns))

740

In [108]:
test_pred = baseline.predict_xgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [109]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_XgbFeatureEngineering6.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.033553
1,100005,0.123969
2,100013,0.035905
3,100028,0.035663
4,100038,0.175384


### mean mean

In [240]:
# consumer_annuity_to_max_installment_ratio
app_train_mean1 = pd.read_csv('./home-credit-default-risk/datasets/train_hc_only_consumer_annuity_to_max_installment_ratio.csv')
app_test_mean1 =  pd.read_csv('./home-credit-default-risk/datasets/test_hc_only_consumer_annuity_to_max_installment_ratio.csv')
app_train_mean1 = app_train_mean1.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
app_test_mean1 = app_test_mean1.rename(columns={'sk_id_curr': 'SK_ID_CURR'})

### installment_payment_ratio_1000_mean_mean
#### consumer_installment_payment_ratio_1000_mean_mean
app_train_mean2 = pd.read_csv('./home-credit-default-risk/datasets/train_hc_only_consumer_installment_payment_ratio_1000_mean_mean.csv')
app_test_mean2 =  pd.read_csv('./home-credit-default-risk/datasets/test_hc_only_consumer_installment_payment_ratio_1000_mean_mean.csv')
app_train_mean2 = app_train_mean2.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
app_test_mean2 = app_test_mean2.rename(columns={'sk_id_curr': 'SK_ID_CURR'})


#### revo_installment_payment_ratio_1000_mean_mean
app_train_mean3 = pd.read_csv('./home-credit-default-risk/datasets/train_hc_only_revo_installment_payment_ratio_1000_mean_mean.csv')
app_test_mean3 =  pd.read_csv('./home-credit-default-risk/datasets/test_hc_only_revo_installment_payment_ratio_1000_mean_mean.csv')
app_train_mean3 = app_train_mean3.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
app_test_mean3 = app_test_mean3.rename(columns={'sk_id_curr': 'SK_ID_CURR'})

#### installment_payment_ratio_1000_mean_mean
app_train_mean4 = pd.read_csv('./home-credit-default-risk/datasets/train_hc_only_cash_installment_payment_ratio_1000_mean_mean.csv')
app_test_mean4 =  pd.read_csv('./home-credit-default-risk/datasets/test_hc_only_cash_installment_payment_ratio_1000_mean_mean.csv')
app_train_mean4 = app_train_mean4.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
app_test_mean4 = app_test_mean4.rename(columns={'sk_id_curr': 'SK_ID_CURR'})


In [241]:
# infをnanへ置換
app_train_mean1 = app_train_mean1.replace([np.inf, -np.inf], np.nan)
app_test_mean1 = app_train_mean1.replace([np.inf, -np.inf], np.nan)

app_train_mean2 = app_train_mean2.replace([np.inf, -np.inf], np.nan)
app_test_mean2 = app_test_mean2.replace([np.inf, -np.inf], np.nan)

app_train_mean3 = app_train_mean3.replace([np.inf, -np.inf], np.nan)
app_test_mean3 = app_test_mean3.replace([np.inf, -np.inf], np.nan)

app_train_mean4 = app_train_mean4.replace([np.inf, -np.inf], np.nan)
app_test_mean4 = app_train_mean4.replace([np.inf, -np.inf], np.nan)

In [242]:
df_train = pd.merge(app_train, bureau_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, installments_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, previous_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, pos_ohe_agg, on='SK_ID_CURR', how='left')

df_train = pd.merge(df_train, app_train_mean1, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, app_train_mean2, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, app_train_mean3, on='SK_ID_CURR', how='left')
df_train = pd.merge(df_train, app_train_mean4, on='SK_ID_CURR', how='left')

print(df_train.shape)
df_train.head()

(307511, 745)


,SK_ID_CURR,TARGET,CODE_GENDER,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_INCOME_TYPE,NAME_FAMILY_STATUS,...,NAME_CONTRACT_STATUS_Returned to the store_mean,NAME_CONTRACT_STATUS_Signed_mean,NAME_CONTRACT_STATUS_XNA_mean,NAME_CONTRACT_STATUS_nan_mean,SK_ID_PREV_count,SK_ID_PREV_nunique,consumer_annuity_to_max_installment_ratio,consumer_installment_payment_ratio_1000_mean_mean,revo_installment_payment_ratio_1000_mean_mean,installment_payment_ratio_1000_mean_mean
0,100002,1,1,0,202500.0,406597.5,24700.5,351000.0,0,1,...,0.000000,0.000000,0.0,0.0,19.0,1.0,-988.0000,0.0,NaN,NaN
1,100003,0,0,0,270000.0,1293502.5,35698.5,1129500.0,3,0,...,0.000000,0.000000,0.0,0.0,28.0,3.0,NaN,0.0,NaN,0.0
2,100004,0,1,0,67500.0,135000.0,6750.0,135000.0,0,1,...,0.000000,0.000000,0.0,0.0,4.0,1.0,NaN,NaN,NaN,NaN
3,100006,0,0,0,135000.0,312682.5,29686.5,297000.0,0,2,...,0.047619,0.000000,0.0,0.0,21.0,3.0,-2698.7273,0.0,NaN,0.0
4,100007,0,1,0,121500.0,513000.0,21865.5,513000.0,0,1,...,0.000000,0.015152,0.0,0.0,66.0,5.0,NaN,NaN,NaN,0.0


In [243]:
x_train = df_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train['TARGET']
id_train = df_train[['SK_ID_CURR']]

In [244]:
baseline = Baseline('SK_ID_CURR')

train_oof, imp, metrics = baseline.train_xgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 743) (61503, 743)
[22:51:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_state", "silent", "subsumple" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-auc:0.73967	validation_1-auc:0.72134
[1]	validation_0-auc:0.75401	validation_1-auc:0.73238
[2]	validation_0-auc:0.76045	validation_1-auc:0.73654
[3]	validation_0-auc:0.76668	validation_1-auc:0.73804
[4]	validation_0-auc:0.76963	validation_1-auc:0.73913
[5]	validation_0-auc:0.77382	validation_1-auc:0.74072
[6]	validation_0-auc:0.77741	validation_1-auc:0.74180
[7]	validation_0-auc:0.77963	validation_1-auc:0.74301
[8]	validation_0-auc:0.78195	validation_1-auc:0.74

[30]	validation_0-auc:0.82636	validation_1-auc:0.76192
[31]	validation_0-auc:0.82797	validation_1-auc:0.76245
[32]	validation_0-auc:0.83017	validation_1-auc:0.76358
[33]	validation_0-auc:0.83233	validation_1-auc:0.76414
[34]	validation_0-auc:0.83426	validation_1-auc:0.76513
[35]	validation_0-auc:0.83597	validation_1-auc:0.76576
[36]	validation_0-auc:0.83813	validation_1-auc:0.76642
[37]	validation_0-auc:0.83993	validation_1-auc:0.76711
[38]	validation_0-auc:0.84144	validation_1-auc:0.76751
[39]	validation_0-auc:0.84342	validation_1-auc:0.76857
[40]	validation_0-auc:0.84531	validation_1-auc:0.76882
[41]	validation_0-auc:0.84688	validation_1-auc:0.76962
[42]	validation_0-auc:0.84862	validation_1-auc:0.76994
[43]	validation_0-auc:0.85045	validation_1-auc:0.77040
[44]	validation_0-auc:0.85231	validation_1-auc:0.77157
[45]	validation_0-auc:0.85406	validation_1-auc:0.77227
[46]	validation_0-auc:0.85541	validation_1-auc:0.77277
[47]	validation_0-auc:0.85684	validation_1-auc:0.77324
[48]	valid

[70]	validation_0-auc:0.88286	validation_1-auc:0.78077
[71]	validation_0-auc:0.88389	validation_1-auc:0.78089
[72]	validation_0-auc:0.88507	validation_1-auc:0.78100
[73]	validation_0-auc:0.88619	validation_1-auc:0.78100
[74]	validation_0-auc:0.88721	validation_1-auc:0.78117
[75]	validation_0-auc:0.88812	validation_1-auc:0.78126
[76]	validation_0-auc:0.88867	validation_1-auc:0.78127
[77]	validation_0-auc:0.88944	validation_1-auc:0.78140
[78]	validation_0-auc:0.89015	validation_1-auc:0.78137
[79]	validation_0-auc:0.89085	validation_1-auc:0.78158
[80]	validation_0-auc:0.89157	validation_1-auc:0.78175
[81]	validation_0-auc:0.89255	validation_1-auc:0.78195
[82]	validation_0-auc:0.89363	validation_1-auc:0.78195
[83]	validation_0-auc:0.89441	validation_1-auc:0.78206
[84]	validation_0-auc:0.89494	validation_1-auc:0.78213
[85]	validation_0-auc:0.89587	validation_1-auc:0.78228
[86]	validation_0-auc:0.89654	validation_1-auc:0.78230
[87]	validation_0-auc:0.89712	validation_1-auc:0.78225
[88]	valid

[0]	validation_0-auc:0.73995	validation_1-auc:0.71684
[1]	validation_0-auc:0.75465	validation_1-auc:0.72696
[2]	validation_0-auc:0.76180	validation_1-auc:0.73274
[3]	validation_0-auc:0.76685	validation_1-auc:0.73611
[4]	validation_0-auc:0.77119	validation_1-auc:0.73816
[5]	validation_0-auc:0.77570	validation_1-auc:0.73976
[6]	validation_0-auc:0.77789	validation_1-auc:0.74056
[7]	validation_0-auc:0.78099	validation_1-auc:0.74153
[8]	validation_0-auc:0.78380	validation_1-auc:0.74264
[9]	validation_0-auc:0.78692	validation_1-auc:0.74476
[10]	validation_0-auc:0.78871	validation_1-auc:0.74474
[11]	validation_0-auc:0.79093	validation_1-auc:0.74559
[12]	validation_0-auc:0.79330	validation_1-auc:0.74617
[13]	validation_0-auc:0.79552	validation_1-auc:0.74681
[14]	validation_0-auc:0.79754	validation_1-auc:0.74720
[15]	validation_0-auc:0.79868	validation_1-auc:0.74812
[16]	validation_0-auc:0.80058	validation_1-auc:0.74866
[17]	validation_0-auc:0.80302	validation_1-auc:0.74976
[18]	validation_0-au

In [245]:
imp.sort_values("imp", ascending=False)[:20]

,col,imp,imp_std
386,EXT_SOURCE_mean,0.020237,0.001274
530,NEW_EXT_SOURCES_GEO,0.017577,0.000641
387,EXT_SOURCE_min,0.013713,0.001581
385,EXT_SOURCE_max,0.007322,0.000695
532,NEW_EXT_SOURCES_MEDIAN,0.006976,0.000528
527,NEW_DOC_IND_KURT,0.004163,0.000336
292,CODE_GENDER,0.004144,0.000136
498,NAME_EDUCATION_TYPE_Higher education,0.004057,0.000407
501,NAME_EDUCATION_TYPE_Secondary / secondary special,0.003955,0.000391
202,ANNUITY_div_CREDIT,0.003367,0.000150


In [246]:
df_test = pd.merge(app_test, bureau_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, installments_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, previous_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, credit_ohe_agg, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, pos_ohe_agg, on='SK_ID_CURR', how='left')

df_test = pd.merge(df_test, app_test_mean1, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, app_test_mean2, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, app_test_mean3, on='SK_ID_CURR', how='left')
df_test = pd.merge(df_test, app_test_mean4, on='SK_ID_CURR', how='left')

# データセット作成
x_test = df_test.drop(columns=['SK_ID_CURR'])
id_test = df_test[['SK_ID_CURR']]

len(list(df_test.columns))

744

In [247]:
test_pred = baseline.predict_xgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [248]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_XgbFeatureEngineering7.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.041751
1,100005,0.127192
2,100013,0.027918
3,100028,0.040672
4,100038,0.151216


### 

## 変数選択

In [249]:
#imp_columns = list(imp.sort_values("imp", ascending=False)['col'])
columns = list(imp.sort_values("imp", ascending=False)['col'][:700])
#print(len(imp_columns))
print(len(columns))

700


In [250]:
if 'SK_ID_CURR' not in columns:
    columns.append('SK_ID_CURR')
if 'TARGET' not in columns:
    columns.append('TARGET')
    
columns

['EXT_SOURCE_mean',
 'NEW_EXT_SOURCES_GEO',
 'EXT_SOURCE_min',
 'EXT_SOURCE_max',
 'NEW_EXT_SOURCES_MEDIAN',
 'NEW_DOC_IND_KURT',
 'CODE_GENDER',
 'NAME_EDUCATION_TYPE_Higher education',
 'NAME_EDUCATION_TYPE_Secondary / secondary special',
 'ANNUITY_div_CREDIT',
 'OCCUPATION_TYPE_Core staff',
 'FLAG_DOCUMENT_3',
 'CNT_DRAWINGS_ATM_CURRENT_mean_y',
 'CREDIT_TYPE_Mortgage_mean',
 'NAME_INCOME_TYPE',
 'app EXT_SOURCE_2_times_EXT_SOURCE_3',
 'NEW_CREDIT_TO_GOODS_RATIO',
 'NEW_CREDIT_TO_ANNUITY_RATIO',
 'app EXT_SOURCE_2_times_DAYS_EMPLOYED',
 'FLAG_OWN_CAR_Y',
 'EXT_SOURCE_3',
 'SOURCES_PROD',
 'CNT_DRAWINGS_CURRENT_mean_y',
 'FLAG_WORK_PHONE',
 'REGION_RATING_CLIENT_W_CITY',
 'credit_goods_price_ratio',
 'AMT_BALANCE_mean_y',
 'CNT_DRAWINGS_CURRENT_mean_x',
 'AMT_RECIVABLE_mean_y',
 'AMT_GOODS_PRICE',
 'credit_downpayment',
 'debt_credit_ratio_None',
 'AMT_CREDIT_SUM_OVERDUE_max',
 'app EXT_SOURCE_1_div_DAYS_BIRTH',
 'consumer_installment_payment_ratio_1000_mean_mean',
 'ORGANIZATION_TYP

In [251]:
df_train_imp = df_train[columns].copy()

if 'TARGET' in columns:
    columns.remove('TARGET')
    
df_test_imp = df_test[columns].copy()

In [252]:
# データセット作成
x_train = df_train_imp.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = df_train_imp['TARGET']
id_train = df_train_imp[['SK_ID_CURR']]

x_train.head()

,EXT_SOURCE_mean,NEW_EXT_SOURCES_GEO,EXT_SOURCE_min,EXT_SOURCE_max,NEW_EXT_SOURCES_MEDIAN,NEW_DOC_IND_KURT,CODE_GENDER,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Secondary / secondary special,ANNUITY_div_CREDIT,...,CREDIT_TYPE_Loan_for_purchase_of_shares_(margin_lending)_mean,SK_DPD_DEF_min_y,SK_DPD_DEF_min_x,SK_DPD_DEF_min,ORGANIZATION_TYPE_Industry: type 2,CREDIT_CURRENCY_currency_4_mean,ORGANIZATION_TYPE_Industry: type 6,CREDIT_CURRENCY_nan_mean,ORGANIZATION_TYPE_Industry: type 8,FLAG_DOCUMENT_15
0,0.161787,0.144914,0.083037,0.262949,0.139376,20.0,1,0,1,0.060749,...,0.0,NaN,NaN,0.0,0,0.0,0,0.0,0,0
1,0.466757,0.440096,0.311267,0.622246,0.466757,20.0,0,1,0,0.027598,...,0.0,NaN,NaN,0.0,0,0.0,0,0.0,0,0
2,0.642739,0.636848,0.555912,0.729567,0.642739,0.0,1,0,1,0.050000,...,0.0,NaN,NaN,0.0,0,0.0,0,0.0,0,0
3,0.650442,0.650442,0.650442,0.650442,0.650442,20.0,0,0,1,0.094941,...,NaN,0.0,0.0,0.0,0,NaN,0,NaN,0,0
4,0.322738,0.322738,0.322738,0.322738,0.322738,20.0,1,0,1,0.042623,...,0.0,NaN,NaN,0.0,0,0.0,0,0.0,0,0


In [253]:
# infをnanへ置換
x_train = x_train.replace([np.inf, -np.inf], np.nan)

In [254]:
baseline = Baseline('SK_ID_CURR')

train_oof, imp, metrics = baseline.train_xgb(x_train
                                    ,y_train
                                    ,id_train
                                    ,params
                                    ,list_nfold=[0,1,2,3,4]
                                    ,n_splits=5)

-------------------- 0 --------------------
(246008, 700) (61503, 700)
[22:53:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_state", "silent", "subsumple" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-auc:0.73940	validation_1-auc:0.72088
[1]	validation_0-auc:0.75433	validation_1-auc:0.73094
[2]	validation_0-auc:0.76176	validation_1-auc:0.73522
[3]	validation_0-auc:0.76822	validation_1-auc:0.73954
[4]	validation_0-auc:0.77138	validation_1-auc:0.74057
[5]	validation_0-auc:0.77429	validation_1-auc:0.74208
[6]	validation_0-auc:0.77721	validation_1-auc:0.74365
[7]	validation_0-auc:0.77941	validation_1-auc:0.74443
[8]	validation_0-auc:0.78132	validation_1-auc:0.74

[30]	validation_0-auc:0.82678	validation_1-auc:0.76270
[31]	validation_0-auc:0.82920	validation_1-auc:0.76306
[32]	validation_0-auc:0.83087	validation_1-auc:0.76395
[33]	validation_0-auc:0.83252	validation_1-auc:0.76488
[34]	validation_0-auc:0.83443	validation_1-auc:0.76549
[35]	validation_0-auc:0.83631	validation_1-auc:0.76600
[36]	validation_0-auc:0.83828	validation_1-auc:0.76639
[37]	validation_0-auc:0.84047	validation_1-auc:0.76734
[38]	validation_0-auc:0.84202	validation_1-auc:0.76773
[39]	validation_0-auc:0.84416	validation_1-auc:0.76835
[40]	validation_0-auc:0.84594	validation_1-auc:0.76881
[41]	validation_0-auc:0.84747	validation_1-auc:0.76932
[42]	validation_0-auc:0.84900	validation_1-auc:0.76959
[43]	validation_0-auc:0.85096	validation_1-auc:0.77019
[44]	validation_0-auc:0.85265	validation_1-auc:0.77088
[45]	validation_0-auc:0.85401	validation_1-auc:0.77158
[46]	validation_0-auc:0.85553	validation_1-auc:0.77197
[47]	validation_0-auc:0.85713	validation_1-auc:0.77248
[48]	valid

[70]	validation_0-auc:0.88296	validation_1-auc:0.77997
[71]	validation_0-auc:0.88403	validation_1-auc:0.78006
[72]	validation_0-auc:0.88512	validation_1-auc:0.78018
[73]	validation_0-auc:0.88617	validation_1-auc:0.78025
[74]	validation_0-auc:0.88720	validation_1-auc:0.78022
[75]	validation_0-auc:0.88836	validation_1-auc:0.78028
[76]	validation_0-auc:0.88929	validation_1-auc:0.78048
[77]	validation_0-auc:0.88957	validation_1-auc:0.78042
[78]	validation_0-auc:0.89047	validation_1-auc:0.78081
[79]	validation_0-auc:0.89137	validation_1-auc:0.78094
[80]	validation_0-auc:0.89193	validation_1-auc:0.78114
[81]	validation_0-auc:0.89299	validation_1-auc:0.78149
[82]	validation_0-auc:0.89365	validation_1-auc:0.78151
[83]	validation_0-auc:0.89433	validation_1-auc:0.78143
[84]	validation_0-auc:0.89549	validation_1-auc:0.78135
[85]	validation_0-auc:0.89621	validation_1-auc:0.78143
[86]	validation_0-auc:0.89695	validation_1-auc:0.78138
[87]	validation_0-auc:0.89790	validation_1-auc:0.78147
[88]	valid

[0]	validation_0-auc:0.74167	validation_1-auc:0.71992
[1]	validation_0-auc:0.75653	validation_1-auc:0.72890
[2]	validation_0-auc:0.76393	validation_1-auc:0.73434
[3]	validation_0-auc:0.76925	validation_1-auc:0.73744
[4]	validation_0-auc:0.77272	validation_1-auc:0.73888
[5]	validation_0-auc:0.77629	validation_1-auc:0.74173
[6]	validation_0-auc:0.77909	validation_1-auc:0.74255
[7]	validation_0-auc:0.78141	validation_1-auc:0.74348
[8]	validation_0-auc:0.78307	validation_1-auc:0.74380
[9]	validation_0-auc:0.78461	validation_1-auc:0.74486
[10]	validation_0-auc:0.78670	validation_1-auc:0.74573
[11]	validation_0-auc:0.78945	validation_1-auc:0.74625
[12]	validation_0-auc:0.79127	validation_1-auc:0.74724
[13]	validation_0-auc:0.79295	validation_1-auc:0.74802
[14]	validation_0-auc:0.79478	validation_1-auc:0.74865
[15]	validation_0-auc:0.79655	validation_1-auc:0.74957
[16]	validation_0-auc:0.79867	validation_1-auc:0.75034
[17]	validation_0-auc:0.80037	validation_1-auc:0.75047
[18]	validation_0-au

In [255]:
# データセット作成
x_test = df_test_imp.drop(columns=['SK_ID_CURR'])
id_test = df_test_imp[['SK_ID_CURR']]

In [256]:
test_pred = baseline.predict_xgb(x_test
                                ,id_test
                                ,list_nfold=[0,1,2,3,4])

-------------------- 0 --------------------
-------------------- 1 --------------------
-------------------- 2 --------------------
-------------------- 3 --------------------
-------------------- 4 --------------------
Done.


In [257]:
df_submit = test_pred.rename(columns={"pred":"TARGET"})
print(df_submit.shape)
display(df_submit.head())
df_submit.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submission_XgbFeatureEngineering8.csv" # 出力先
    ,index=None                                        # indexの出力有無
)

(48744, 2)


,SK_ID_CURR,TARGET
0,100001,0.038214
1,100005,0.123099
2,100013,0.030748
3,100028,0.036872
4,100038,0.157489


In [258]:
print(np.log(0.1))
print(np.log(10))

-2.3025850929940455
2.302585092994046


## パラメータチューニング

In [259]:
#baseline = Baseline('SK_ID_CURR')

#sampler = optuna.samplers.TPESampler(seed=123)
#study = optuna.create_study(sampler=sampler, direction='maximize')
#study.optimize(objective, n_trials=50, n_jobs=5)

[I 2022-06-14 22:54:36,391] A new study created in memory with name: no-name-1657df8c-8c94-4b88-b574-b2c44474580d


[0]	validation_0-auc:0.73940	validation_1-auc:0.72088
[0]	validation_0-auc:0.73940	validation_1-auc:0.72088
[0]	validation_0-auc:0.73940	validation_1-auc:0.72088
[0]	validation_0-auc:0.73940	validation_1-auc:0.72088
[0]	validation_0-auc:0.73940	validation_1-auc:0.72088
[1]	validation_0-auc:0.75433	validation_1-auc:0.73094
[1]	validation_0-auc:0.75433	validation_1-auc:0.73094
[1]	validation_0-auc:0.75433	validation_1-auc:0.73094
[1]	validation_0-auc:0.75433	validation_1-auc:0.73094
[1]	validation_0-auc:0.75433	validation_1-auc:0.73094
[2]	validation_0-auc:0.76176	validation_1-auc:0.73522
[2]	validation_0-auc:0.76176	validation_1-auc:0.73522
[2]	validation_0-auc:0.76176	validation_1-auc:0.73522
[2]	validation_0-auc:0.76176	validation_1-auc:0.73522
[2]	validation_0-auc:0.76176	validation_1-auc:0.73522
[3]	validation_0-auc:0.76822	validation_1-auc:0.73954
[3]	validation_0-auc:0.76822	validation_1-auc:0.73954
[3]	validation_0-auc:0.76822	validation_1-auc:0.73954
[3]	validation_0-auc:0.76822

[29]	validation_0-auc:0.82516	validation_1-auc:0.75985
[30]	validation_0-auc:0.82705	validation_1-auc:0.76027
[31]	validation_0-auc:0.82911	validation_1-auc:0.76068
[30]	validation_0-auc:0.82705	validation_1-auc:0.76027
[30]	validation_0-auc:0.82705	validation_1-auc:0.76027
[30]	validation_0-auc:0.82705	validation_1-auc:0.76027
[31]	validation_0-auc:0.82911	validation_1-auc:0.76068
[32]	validation_0-auc:0.83111	validation_1-auc:0.76140
[31]	validation_0-auc:0.82911	validation_1-auc:0.76068
[31]	validation_0-auc:0.82911	validation_1-auc:0.76068
[31]	validation_0-auc:0.82911	validation_1-auc:0.76068
[32]	validation_0-auc:0.83111	validation_1-auc:0.76140
[33]	validation_0-auc:0.83316	validation_1-auc:0.76203
[32]	validation_0-auc:0.83111	validation_1-auc:0.76140
[32]	validation_0-auc:0.83111	validation_1-auc:0.76140
[32]	validation_0-auc:0.83111	validation_1-auc:0.76140
[33]	validation_0-auc:0.83316	validation_1-auc:0.76203
[34]	validation_0-auc:0.83488	validation_1-auc:0.76240
[33]	valid

[62]	validation_0-auc:0.87380	validation_1-auc:0.77466
[59]	validation_0-auc:0.87087	validation_1-auc:0.77384
[59]	validation_0-auc:0.87087	validation_1-auc:0.77384
[60]	validation_0-auc:0.87180	validation_1-auc:0.77414
[63]	validation_0-auc:0.87465	validation_1-auc:0.77474
[60]	validation_0-auc:0.87180	validation_1-auc:0.77414
[59]	validation_0-auc:0.87087	validation_1-auc:0.77384
[60]	validation_0-auc:0.87180	validation_1-auc:0.77414
[61]	validation_0-auc:0.87268	validation_1-auc:0.77445
[64]	validation_0-auc:0.87561	validation_1-auc:0.77498
[61]	validation_0-auc:0.87268	validation_1-auc:0.77445
[60]	validation_0-auc:0.87180	validation_1-auc:0.77414
[61]	validation_0-auc:0.87268	validation_1-auc:0.77445
[62]	validation_0-auc:0.87380	validation_1-auc:0.77466
[65]	validation_0-auc:0.87674	validation_1-auc:0.77517
[62]	validation_0-auc:0.87380	validation_1-auc:0.77466
[61]	validation_0-auc:0.87268	validation_1-auc:0.77445
[62]	validation_0-auc:0.87380	validation_1-auc:0.77466
[63]	valid

[89]	validation_0-auc:0.89759	validation_1-auc:0.77868
[89]	validation_0-auc:0.89759	validation_1-auc:0.77868
[93]	validation_0-auc:0.90022	validation_1-auc:0.77887
[87]	validation_0-auc:0.89572	validation_1-auc:0.77850
[90]	validation_0-auc:0.89841	validation_1-auc:0.77876
[90]	validation_0-auc:0.89841	validation_1-auc:0.77876
[90]	validation_0-auc:0.89841	validation_1-auc:0.77876
[94]	validation_0-auc:0.90115	validation_1-auc:0.77893
[88]	validation_0-auc:0.89665	validation_1-auc:0.77856
[91]	validation_0-auc:0.89869	validation_1-auc:0.77880
[91]	validation_0-auc:0.89869	validation_1-auc:0.77880
[95]	validation_0-auc:0.90154	validation_1-auc:0.77895
[91]	validation_0-auc:0.89869	validation_1-auc:0.77880
[89]	validation_0-auc:0.89759	validation_1-auc:0.77868
[96]	validation_0-auc:0.90204	validation_1-auc:0.77906
[92]	validation_0-auc:0.89948	validation_1-auc:0.77885
[92]	validation_0-auc:0.89948	validation_1-auc:0.77885
[92]	validation_0-auc:0.89948	validation_1-auc:0.77885
[90]	valid

...

[34]	validation_0-auc:0.83488	validation_1-auc:0.76240
[46]	validation_0-auc:0.85477	validation_1-auc:0.76889
[35]	validation_0-auc:0.83658	validation_1-auc:0.76283
[35]	validation_0-auc:0.83658	validation_1-auc:0.76283
[35]	validation_0-auc:0.83658	validation_1-auc:0.76283
[47]	validation_0-auc:0.85617	validation_1-auc:0.76936
[36]	validation_0-auc:0.83903	validation_1-auc:0.76357
[36]	validation_0-auc:0.83903	validation_1-auc:0.76357
[48]	validation_0-auc:0.85741	validation_1-auc:0.76983
[36]	validation_0-auc:0.83903	validation_1-auc:0.76357
[37]	validation_0-auc:0.84064	validation_1-auc:0.76440
[37]	validation_0-auc:0.84064	validation_1-auc:0.76440
[49]	validation_0-auc:0.85879	validation_1-auc:0.77023
[37]	validation_0-auc:0.84064	validation_1-auc:0.76440
[38]	validation_0-auc:0.84266	validation_1-auc:0.76450
[50]	validation_0-auc:0.86014	validation_1-auc:0.77066
[38]	validation_0-auc:0.84266	validation_1-auc:0.76450
[38]	validation_0-auc:0.84266	validation_1-auc:0.76450
[39]	valid

[86]	validation_0-auc:0.89499	validation_1-auc:0.77857
[71]	validation_0-auc:0.88273	validation_1-auc:0.77671
[70]	validation_0-auc:0.88161	validation_1-auc:0.77652
[72]	validation_0-auc:0.88330	validation_1-auc:0.77691
[87]	validation_0-auc:0.89572	validation_1-auc:0.77850
[72]	validation_0-auc:0.88330	validation_1-auc:0.77691
[73]	validation_0-auc:0.88440	validation_1-auc:0.77706
[71]	validation_0-auc:0.88273	validation_1-auc:0.77671
[88]	validation_0-auc:0.89665	validation_1-auc:0.77856
[74]	validation_0-auc:0.88484	validation_1-auc:0.77705
[73]	validation_0-auc:0.88440	validation_1-auc:0.77706
[72]	validation_0-auc:0.88330	validation_1-auc:0.77691
[89]	validation_0-auc:0.89759	validation_1-auc:0.77868
[74]	validation_0-auc:0.88484	validation_1-auc:0.77705
[75]	validation_0-auc:0.88576	validation_1-auc:0.77711
[73]	validation_0-auc:0.88440	validation_1-auc:0.77706
[90]	validation_0-auc:0.89841	validation_1-auc:0.77876
[75]	validation_0-auc:0.88576	validation_1-auc:0.77711
[76]	valid

[I 2022-06-14 23:06:33,853] Trial 46 finished with value: 0.7792979826916042 and parameters: {'min_child_weight': 0.43865357766541185, 'max_depth': 6, 'subsample': 0.8, 'colsample_bytree': 0.85, 'colsample_bylevel': 0.30000000000000004, 'gamma': 1.0778671351412035e-07, 'alpha': 4.414980453505299e-07, 'lambda': 0.6236173360081361}. Best is trial 3 with value: 0.7792979826916042.


[85]	validation_0-auc:0.89373	validation_1-auc:0.77834
[84]	validation_0-auc:0.89259	validation_1-auc:0.77823
[87]	validation_0-auc:0.89572	validation_1-auc:0.77850
[86]	validation_0-auc:0.89499	validation_1-auc:0.77857
[85]	validation_0-auc:0.89373	validation_1-auc:0.77834
[87]	validation_0-auc:0.89572	validation_1-auc:0.77850
[88]	validation_0-auc:0.89665	validation_1-auc:0.77856
[86]	validation_0-auc:0.89499	validation_1-auc:0.77857
[89]	validation_0-auc:0.89759	validation_1-auc:0.77868
[88]	validation_0-auc:0.89665	validation_1-auc:0.77856
[87]	validation_0-auc:0.89572	validation_1-auc:0.77850
[90]	validation_0-auc:0.89841	validation_1-auc:0.77876
[89]	validation_0-auc:0.89759	validation_1-auc:0.77868
[88]	validation_0-auc:0.89665	validation_1-auc:0.77856
[91]	validation_0-auc:0.89869	validation_1-auc:0.77880
[90]	validation_0-auc:0.89841	validation_1-auc:0.77876
[89]	validation_0-auc:0.89759	validation_1-auc:0.77868
[91]	validation_0-auc:0.89869	validation_1-auc:0.77880
[92]	valid

[I 2022-06-14 23:06:38,384] Trial 47 finished with value: 0.7792979826916042 and parameters: {'min_child_weight': 0.44687983683714916, 'max_depth': 6, 'subsample': 0.75, 'colsample_bytree': 0.85, 'colsample_bylevel': 0.30000000000000004, 'gamma': 1.4674129596675382e-07, 'alpha': 5.127426656391053e-07, 'lambda': 0.5543080763648722}. Best is trial 3 with value: 0.7792979826916042.
[I 2022-06-14 23:06:38,570] Trial 48 finished with value: 0.7792979826916042 and parameters: {'min_child_weight': 6.846164105498921, 'max_depth': 4, 'subsample': 0.75, 'colsample_bytree': 0.85, 'colsample_bylevel': 0.35, 'gamma': 1.915028866594826e-06, 'alpha': 4.969165695677602e-07, 'lambda': 0.2893041660749811}. Best is trial 3 with value: 0.7792979826916042.
[I 2022-06-14 23:06:39,102] Trial 49 finished with value: 0.7792979826916042 and parameters: {'min_child_weight': 2.380664736982173, 'max_depth': 4, 'subsample': 0.75, 'colsample_bytree': 0.9, 'colsample_bylevel': 0.35, 'gamma': 3.9984503147983453e-07, '

In [260]:
#trial = study.best_trial
#print("auc(best)={:.4f}".format(trial.value))
#display(trial.params)

auc(best)=0.7793


{'min_child_weight': 3.2762615507173805,
 'max_depth': 4,
 'subsample': 0.7000000000000001,
 'colsample_bytree': 0.75,
 'colsample_bylevel': 0.8,
 'gamma': 2.6285548524681706e-06,
 'alpha': 0.4564905434359725,
 'lambda': 0.17868395970497722}

In [261]:
#params_best = trial.params
#params_best.update(params_base)
#display(params_best)

{'min_child_weight': 1,
 'max_depth': 5,
 'subsample': 0.8,
 'colsample_bytree': 0.8,
 'colsample_bylevel': 0.3,
 'gamma': 0.0,
 'alpha': 0.0,
 'lambda': 1.0,
 'booster': 'gbtree',
 'objective': 'binary:logistic',
 'eta': 0.1,
 'random_state': 17,
 'gpu_id': 0,
 'tree_method': 'gpu_hist',
 'eval_metric': 'auc',
 'num_round': 100000}

In [262]:
#train_oof, imp, metrics = baseline.train_xgb(x_train
#                                    ,y_train
#                                    ,id_train
#                                    ,list_nfold=[0,1,2,3,4]
#                                    ,n_splits=5
#                                    ,params=params_best)

-------------------- 0 --------------------
(246008, 700) (61503, 700)
[23:06:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "num_round" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-auc:0.72159	validation_1-auc:0.71514
[1]	validation_0-auc:0.73053	validation_1-auc:0.72237
[2]	validation_0-auc:0.73617	validation_1-auc:0.72854
[3]	validation_0-auc:0.73828	validation_1-auc:0.72928
[4]	validation_0-auc:0.74089	validation_1-auc:0.73125
[5]	validation_0-auc:0.74185	validation_1-auc:0.73241
[6]	validation_0-auc:0.74274	validation_1-auc:0.73286
[7]	validation_0-auc:0.74389	validation_1-auc:0.73391
[8]	validation_0-auc:0.74513	validation_1-auc:0.73488
[9]	validation_0-auc:0.74

[32]	validation_0-auc:0.76585	validation_1-auc:0.75268
[33]	validation_0-auc:0.76682	validation_1-auc:0.75337
[34]	validation_0-auc:0.76793	validation_1-auc:0.75415
[35]	validation_0-auc:0.76889	validation_1-auc:0.75477
[36]	validation_0-auc:0.77008	validation_1-auc:0.75579
[37]	validation_0-auc:0.77087	validation_1-auc:0.75619
[38]	validation_0-auc:0.77211	validation_1-auc:0.75741
[39]	validation_0-auc:0.77325	validation_1-auc:0.75816
[40]	validation_0-auc:0.77432	validation_1-auc:0.75883
[41]	validation_0-auc:0.77551	validation_1-auc:0.75997
[42]	validation_0-auc:0.77645	validation_1-auc:0.76074
[43]	validation_0-auc:0.77754	validation_1-auc:0.76160
[44]	validation_0-auc:0.77856	validation_1-auc:0.76271
[45]	validation_0-auc:0.77942	validation_1-auc:0.76333
[46]	validation_0-auc:0.78039	validation_1-auc:0.76403
[47]	validation_0-auc:0.78116	validation_1-auc:0.76430
[48]	validation_0-auc:0.78196	validation_1-auc:0.76494
[49]	validation_0-auc:0.78254	validation_1-auc:0.76539
[50]	valid

[72]	validation_0-auc:0.79742	validation_1-auc:0.77679
[73]	validation_0-auc:0.79811	validation_1-auc:0.77716
[74]	validation_0-auc:0.79856	validation_1-auc:0.77732
[75]	validation_0-auc:0.79905	validation_1-auc:0.77752
[76]	validation_0-auc:0.79937	validation_1-auc:0.77769
[77]	validation_0-auc:0.79992	validation_1-auc:0.77802
[78]	validation_0-auc:0.80036	validation_1-auc:0.77831
[79]	validation_0-auc:0.80088	validation_1-auc:0.77860
[80]	validation_0-auc:0.80135	validation_1-auc:0.77886
[81]	validation_0-auc:0.80181	validation_1-auc:0.77918
[82]	validation_0-auc:0.80216	validation_1-auc:0.77921
[83]	validation_0-auc:0.80264	validation_1-auc:0.77936
[84]	validation_0-auc:0.80307	validation_1-auc:0.77959
[85]	validation_0-auc:0.80340	validation_1-auc:0.77971
[86]	validation_0-auc:0.80383	validation_1-auc:0.77979
[87]	validation_0-auc:0.80423	validation_1-auc:0.77988
[88]	validation_0-auc:0.80469	validation_1-auc:0.77988
[89]	validation_0-auc:0.80509	validation_1-auc:0.77997
[90]	valid

[3]	validation_0-auc:0.73917	validation_1-auc:0.72712
[4]	validation_0-auc:0.74128	validation_1-auc:0.72918
[5]	validation_0-auc:0.74245	validation_1-auc:0.73015
[6]	validation_0-auc:0.74359	validation_1-auc:0.73134
[7]	validation_0-auc:0.74478	validation_1-auc:0.73235
[8]	validation_0-auc:0.74590	validation_1-auc:0.73347
[9]	validation_0-auc:0.74710	validation_1-auc:0.73459
[10]	validation_0-auc:0.74759	validation_1-auc:0.73459
[11]	validation_0-auc:0.74779	validation_1-auc:0.73464
[12]	validation_0-auc:0.74829	validation_1-auc:0.73481
[13]	validation_0-auc:0.74955	validation_1-auc:0.73609
[14]	validation_0-auc:0.75007	validation_1-auc:0.73651
[15]	validation_0-auc:0.75120	validation_1-auc:0.73768
[16]	validation_0-auc:0.75219	validation_1-auc:0.73872
[17]	validation_0-auc:0.75246	validation_1-auc:0.73890
[18]	validation_0-auc:0.75355	validation_1-auc:0.73965
[19]	validation_0-auc:0.75444	validation_1-auc:0.73999
[20]	validation_0-auc:0.75514	validation_1-auc:0.74070
[21]	validation_0

In [263]:
### 推論用データセットの作成
#x_test = df_test.drop(columns=['SK_ID_CURR'])
#id_test = df_test[['SK_ID_CURR']]

### カテゴリ変数をcategory型へ変換
#for col in x_test.columns:
#    if x_test[col].dtype=="O":
#        x_test[col] = x_test[col].astype('category')

### predict
#test_pred = baseline.predict_xgb(x_test
#                       ,id_test
#                       ,list_nfold=[0,1,2,3,4])

### make submission-file
#df_submit = test_pred.rename(columns={"pred":"TARGET"})
#print(df_submit.shape)
#display(df_submit.head())
#df_submit.to_csv("./home-credit-default-risk/submit/submission_XgbHyperParameterTuning.csv", index=None)

-------------------- 0 --------------------


ValueError: Feature shape mismatch, expected: 700, got 743